<a href="https://colab.research.google.com/github/hsching/Course-Enrollment-System/blob/main/ISOM3260.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package and connect to Oracle Database

In [ ]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ln -s /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so
! ldconfig


# -- Include cx_Oracle package --
! pip install cx_Oracle


--2023-05-02 10:43:11--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 173.223.108.89
Connecting to download.oracle.com (download.oracle.com)|173.223.108.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M  79.7MB/s    in 0.9s    

2023-05-02 10:43:12 (79.7 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

mv: cannot move 'instantclient_19_6' to '/opt/oracle/instantclient_19_6': Directory not empty
ln: failed to create symbolic link '/usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.0.so': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import cx_Oracle package and connect to Oracle Database
import cx_Oracle

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "LA205"
PASSWORD = "KM534"

dsn_tns = cx_Oracle.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME) 
conn = cx_Oracle.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns) 

c = conn.cursor()

# GUI

In [ ]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import math
import pandas as pd
import re
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import Password
from ipywidgets import DatePicker
from ipywidgets import Dropdown
from ipywidgets import IntText
from ipywidgets import Label
from ipywidgets import HTML
from ipywidgets import Layout
from datetime import date
from datetime import datetime

# frequently used elements
spacer = Label("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------", layout={'height':'20px'})
split = Label("*********************************************************************************************************************************")
space = Label('', layout={'flex': '2'})

# declare global variables
global id, username, password
id = 0

###########################################################
##################### Pages flow design (Cathy)#####################
###########################################################

tb_first = TabBar(['Login', 'Register', "Forgot Password", "Student Home", "Instructor Home", "Manager Home"])

# 2nd: student layer
with tb_first.output_to('Student Home'):
  tb_stu = TabBar(['Account Information', 'Course'])
  with tb_stu.output_to('Course'):
    tb_stu_course = TabBar(['Course Catalog', 'Bookmarks', 'Enrollment Cart', 'View Enrollment', 'SFQ'])
    with tb_stu_course.output_to('Course Catalog'):
      display()

# 2nd: instructor layer
with tb_first.output_to('Instructor Home'):
  tb_ins = TabBar(['Account Information', 'Course'])
  with tb_ins.output_to('Course'):
    tb_ins_course = TabBar(['Course', 'Class Roster', 'Review']) #click button of each course

# 2nd: manager layer
with tb_first.output_to('Manager Home'):
  tb_m = TabBar(['Dashboard', 'Account Maintenance', 'Course', 'Refund List', 'Feedback'])
  with tb_m.output_to('Account Maintenance'):
    tb_m_acmaintain = TabBar(['Update/Delete Student', 'Insert Student', 'Update/Delete Instructor', 'Insert Instructor'])
  with tb_m.output_to('Course'):
    tb_m_course = TabBar(['View Course', 'Update/Delete Course', 'Insert Course', 'Generate Roster'])
  with tb_m.output_to('Feedback'):
    tb_m_feedback = TabBar(['View Feedback'])

def showPage(tabName):
  with tb_first.output_to(tabName):
    display()


def available_course_code(table):
  course_code_options = []
  c.execute("SELECT DISTINCT SUBSTR(course_code, 1, 4) FROM "+ table +" WHERE COURSE_STATUS = 'open'")
  for row in c:
    course_code_options.append(row[0])
  return sorted(course_code_options)

def enrollment_course_code(id):
  enrollment_cart_course_code_options = []
  c.execute("""SELECT DISTINCT course_code 
  FROM course 
  WHERE COURSE_STATUS = 'open' 
  and course_enddate > TO_DATE(SYSDATE, 'DD-MON-YY')
  and course_code not in (select c.course_code from enrollment e
  inner join course c on c.course_id = e.course_id
  where e.stu_id =""" + str(id) + """)""")
  for row in c:
    enrollment_cart_course_code_options.append(row[0])
  return sorted(enrollment_cart_course_code_options)

def bookmark_course_code(id):
  bookmark_course_code_options = []
  bookmark_course_id_options = []
  c.execute("""SELECT b.course_id, c.course_code
    from bookmark b 
    inner join course c
    on b.course_id = c.course_id
    where b.stu_id = """ + str(id))
  for row in c:
    bookmark_course_id_options.append(row[0])
    bookmark_course_code_options.append(row[1])
  return bookmark_course_code_options

def bookmark_course_code_1(id):
  bookmark_course_code_options = []
  bookmark_course_id_options = []
  c.execute("""SELECT b.course_id, c.course_code
    from bookmark b 
    inner join course c
    on b.course_id = c.course_id
    where b.stu_id = """ + str(id))
  for row in c:
    bookmark_course_id_options.append(row[0])
    bookmark_course_code_options.append(row[1])
  return bookmark_course_id_options

##### define login information for manager #######
manager_acc_name = "manager"
manager_acc_pwd = "man"



# tb_first

##########################################################
################ Login Tab - Begin (Cathy) ################
##########################################################
with tb_first.output_to('Login'):

# create 1 label 
  no_account_label = Label("Do not have an account?")
  login_error_msg_label = Label('')
  global_id_label = Label('') #just in case it didn't be stored due to unknown reason

# create 5 button
  login_username_textbox = Text(placeholder='Username:', disabled=False)
  login_password_textbox = Password(placeholder='Password:', disabled=False)


  forgot_password_btn = widgets.Button(description='Forgot Password?')
  login_btn = widgets.Button(description="Log in")
  
  register_as_student_btn = widgets.Button(description="Register as a Student")

# define the event handlers for the two buttons when they are being clicked
  def on_forgot_password_btn_clicked(b):
    showPage("Forgot Password")
  forgot_password_btn.on_click(on_forgot_password_btn_clicked)
  
  def on_login_btn_clicked(b):
    # declare global variables
    global id, username, pwd
    
    # reset value of login_error_msg_label and pwd
    login_error_msg_label.value = ''
    pwd = ''
    
    # verify there's value inside the textbox
    if login_username_textbox.value == '' or login_password_textbox.value == '':
      login_error_msg_label.value = 'Please input valid username and password.'
      return

    if login_username_textbox.value == manager_acc_name:
      if login_password_textbox.value == manager_acc_pwd:
        login_error_msg_label.value = 'Login Successfully.'
        time.sleep(0.5)
        showPage("Manager Home")  
      else:
        login_password_textbox.value = ''
        login_error_msg_label.value = 'Wrong password. Please try again.'
        return
  
    #retrieve info of either instructor
    c.execute("SELECT Stu_ID, Stu_username, Stu_pwd FROM STUDENT WHERE Stu_username = '" + login_username_textbox.value + "'")
    for row in c:
      id = row[0]
      global_id_label.value = str(row[0]) #backup
      username = row[1]
      pwd = row[2]
    if login_password_textbox.value == pwd:
      login_error_msg_label.value = 'Login Successfully.'
      get_acc_info_function()
      get_course_id_for_sfq_function()
      time.sleep(0.5)
      showPage("Student Home")
      return
    elif pwd=='':
      #retrieve info of either instructor
      c.execute("SELECT Ins_ID, Ins_username, Ins_pwd FROM INSTRUCTOR WHERE Ins_username = '" + login_username_textbox.value + "'")
      for row in c:
        id = row[0]
        global_id_label.value = str(row[0]) #backup
        username = row[1]
        pwd = row[2]
      if login_password_textbox.value == pwd:
        login_error_msg_label.value = 'Login Successfully.'
        INST_COURSE_GET()
        INST_INFO_GET()
        time.sleep(0.5)
        showPage("Instructor Home")     
      else:
        login_password_textbox.value = ''
        login_error_msg_label.value = 'Wrong password. Please try again.'
    else:
      login_password_textbox.value = ''
      login_error_msg_label.value = 'Wrong password. Please try again.'

    if pwd=='':
      login_username_textbox.value = ''
      login_password_textbox.value = ''
      login_error_msg_label.value = 'Username not found. Please try again.'
  login_btn.on_click(on_login_btn_clicked)

  def on_register_as_studnet_btn_clicked(b):
    showPage("Register")
  register_as_student_btn.on_click(on_register_as_studnet_btn_clicked)

# display the objects
  forgot_login_btn = widgets.HBox([forgot_password_btn, login_btn])
  reg_label_btn = widgets.HBox([no_account_label, register_as_student_btn])
  display(widgets.VBox([login_username_textbox, login_password_textbox, forgot_login_btn, login_error_msg_label, reg_label_btn]))
################ Login Tab - End ################

###########################################################
################ Register Tab - Begin (Susanna) ################
###########################################################
with tb_first.output_to('Register'):

# create labels 
  Maintain_details_label = Label("Please input your username and password.")
  Rule_label = Label("* must be filled")
  register_status_label = Label("")

  username_full_list = []
  c.execute("""SELECT max(STU_ID) FROM STUDENT""")
  for row in c:
    stu_id_max = row[0]

  reg_id_textbox = Text(description='Student ID: ', value= str(stu_id_max+1), disabled=True)
  reg_name_textbox = Text(description='Name*: ', value='', placeholder='Name', disabled=False)
  reg_username_textbox = Text(description='Username*: ', value='', placeholder='less than 20 characters', disabled=False)
  reg_password_textbox = Password(description='Password*: ', value='', placeholder='less than 20 characters', disabled=False)
  reg_email_textbox = Text(description='Email* : ', value='', placeholder='@gmail.com / @...', disabled=False)
  reg_address_textbox = Text(description='Address*: ', value='', placeholder='less than 50 characters', disabled=False)
  reg_job_textbox = Text(description='Job: ', value='', placeholder='less than 50 characters', disabled=False)
  reg_qualification_textbox = Text(description='Qualification: ', value='', placeholder='less than 50 characters', disabled=False)

  reg_btn = widgets.Button(description="Register")
  
  reg_output = widgets.Output()

  def on_reg_btn_clicked(b):
    reg_output.clear_output()
    error_msg = "Please follow the rules below: "
    c.execute("select stu_username from student")
    for row in c:
      username_full_list.append(row[0])

    if len(reg_name_textbox.value) == 0:
      error_msg += "\nname cannot be empty"

    if reg_username_textbox.value in username_full_list:
      error_msg += """\nusername is occupied"""
    elif len(reg_username_textbox.value) > 20:
      error_msg += """\ninput username of less than 20 characters"""
    elif len(reg_username_textbox.value) == 0:
      error_msg += """\nusername cannot be empty"""
    
    if len(reg_password_textbox.value) > 20:
      error_msg += """\ninput password of less than 20 characters"""
    elif len(reg_password_textbox.value) == 0:
      error_msg += """\npassword cannot be empty"""
    
    if len(reg_email_textbox.value) == 0:
      error_msg += """\nemail cannot be empty"""
    elif "@" not in reg_email_textbox.value or "." not in reg_email_textbox.value:
      error_msg += """\ninput email address with '@gmail.com'/@..."""
    
    if len(reg_address_textbox.value) > 50:
      error_msg += """\ninput address of less than 50 characters"""
    elif len(reg_address_textbox.value) == 0:
      error_msg += """\naddress cannot be empty"""
    
    if len(reg_job_textbox.value) > 50:
      error_msg += """\ninput job title of less than 50 characters"""
    
    if len(reg_qualification_textbox.value) > 50:
      error_msg += """\ninput qualification description of less than 50 characters"""
    
    if len(error_msg) == len("Please follow the rules below: "):
      c.execute("""INSERT INTO STUDENT VALUES ('{}', '{}', '{}','{}', '{}','{}', '{}', '{}')""".format(reg_id_textbox.value, reg_name_textbox.value, reg_username_textbox.value, 
                                        reg_password_textbox.value, reg_email_textbox.value, reg_address_textbox.value, reg_job_textbox.value, reg_qualification_textbox.value))
      c.execute("commit")
      reg_btn.disabled = True
      register_status_label.value = "Register success"
      time.sleep(0.5)
      showPage("Login")
    else:
      with reg_output:
        print(error_msg)
      
  reg_btn.on_click(on_reg_btn_clicked)

  display(widgets.VBox([Maintain_details_label, Rule_label, reg_id_textbox, reg_name_textbox, reg_username_textbox, reg_password_textbox, reg_email_textbox, reg_address_textbox, reg_job_textbox, reg_qualification_textbox, reg_btn, register_status_label, reg_output]))
################ Register Tab - End ################

############################################################
################ Forgot/Reset Password Tab - Begin (Cathy) ################
############################################################
with tb_first.output_to('Forgot Password'):

# create label and textbox
  retrieve_reset_page_label = Label("Input your Name and retrieve your user information here...")
  reset_label = Label('Here is your login information. You can edit directly and click the "Reset" button to update.')
  id_not_found_label = Label('')
  username_error_updated_label = Label('')
  table_name_label = Label('')

  id_textbox = Text(placeholder='Student/Staff ID:', disabled=False)
  reset_username_textbox = Text(description='Username:', disabled=False)
  reset_password_textbox = Text(description='Password:', disabled=False)

# create 3 buttons
  back_to_login_btn = widgets.Button(description='Login with Existing Account', layout={'width':'200px'})
  retrieve_login_info_btn = widgets.Button(description="Retrieve Login Info")
  reset_login_info_btn = widgets.Button(description='Reset')

  def on_back_to_login_btn(b):
    showPage('Login')
  back_to_login_btn.on_click(on_back_to_login_btn)

  def on_retrieve_login_info_btn_clicked(b):
    id_not_found_label.value = ''
    username_error_updated_label.value = ''
    reset_username_textbox.value = ''
    reset_password_textbox.value = ''
    table_name_label.value = ''
    username = ''
    pwd =''
    # test the input value's dtype before processing
    try:
      int(id_textbox.value)
    except ValueError:
      id_not_found_label.value = 'Please input a number.'
      return
    # retrieve login info
    c.execute("SELECT Stu_username, Stu_pwd FROM STUDENT WHERE Stu_id = '" + id_textbox.value + "'")
    for row in c:
      table_name_label.value = 'STUDENT'
      username = row[0]
      pwd = row[1]  
    if pwd == '':
      c.execute("SELECT Ins_username, Ins_pwd FROM INSTRUCTOR WHERE Ins_id = '" + id_textbox.value + "'") #retrieve info of either student or instructor.
      for row in c:
        table_name_label.value = 'INSTRUCTOR'
        username = row[0]
        pwd = row[1]
      if pwd == '':
        id_textbox.value = ''
        id_not_found_label.value = 'ID not found. Please try again.'
        return
    reset_username_textbox.value = username
    reset_password_textbox.value = pwd
  retrieve_login_info_btn.on_click(on_retrieve_login_info_btn_clicked)       

  def on_reset_login_info_btn_clicked(b):
    username_error_updated_label.value = ''
    if table_name_label.value == 'STUDENT':
      c.execute("SELECT Stu_username FROM student where Stu_username = '"+ reset_username_textbox.value +"'")
      if c.fetchall():
        username_error_updated_label.value = 'This username has been taken.'
        return
      c.execute("""UPDATE STUDENT SET Stu_username='{}', Stu_pwd='{}' WHERE Stu_id = '{}' 
      """.format(reset_username_textbox.value, reset_password_textbox.value, id_textbox.value))
      c.execute("commit")
    elif table_name_label.value =='INSTRUCTOR':
      c.execute(
      """UPDATE INTO INSTRUCTOR SET Ins_username='{}', Ins_pwd='{}' WHERE Ins_id = '{}' 
      """.format(reset_username_textbox.value, reset_password_textbox.value, id_textbox.value))
      c.execute("commit")
    username_error_updated_label.value = 'Your login information has been updated successfully.'
  reset_login_info_btn.on_click(on_reset_login_info_btn_clicked)
  
  retrieve_info_textbox_btn = widgets.HBox([id_textbox, retrieve_login_info_btn])
  back_error_msg_label = widgets.HBox([back_to_login_btn, id_not_found_label])
  login_info_textbox = widgets.HBox([reset_username_textbox, reset_password_textbox]) 
  reset_btn_msg_label = widgets.HBox([reset_login_info_btn, username_error_updated_label])
  display(widgets.VBox([retrieve_reset_page_label, retrieve_info_textbox_btn, back_error_msg_label, spacer, reset_label, login_info_textbox, reset_btn_msg_label]))
################ Forgot Password Tab - End ################

# tb_stu

###########################################################
################ Account Information Tab - Begin （Susanna）################
###########################################################
with tb_stu.output_to('Account Information'):
# create 1 label 
  updatepage_label = Label("Update your user name and password here")
  update_status_label = Label("")

# create textboxes
  UPD_INFO_layout = Layout(width='160px')
  # sorry i could not retrieve any info with global_id_label.value here, so i changed it back to str(id)
  Info_id_textbox = Text(disabled=True)
  UPD_name_textbox = Text(disabled=False)
  UPD_username_textbox = Text(disabled=False)
  UPD_password_textbox = Text(disabled=False)
  UPD_email_textbox = Text(disabled=False)
  UPD_address_textbox = Text(disabled=False)
  UPD_job_textbox = Text(disabled=False)
  UPD_qualification_textbox = Text(disabled=False)

  #create labels
  Info_id_label = Label('Student ID:', layout=UPD_INFO_layout)
  UPD_INFO_name_label = Label('Name:', layout=UPD_INFO_layout)
  UPD_INFO_username_label = Label('Username:', layout=UPD_INFO_layout)
  UPD_INFO_email_label = Label('Email:', layout=UPD_INFO_layout)
  UPD_INFO_address_label = Label('Address:', layout=UPD_INFO_layout)
  UPD_INFO_job_label = Label('Job:', layout=UPD_INFO_layout)
  UPD_INFO_quali_label = Label('Qualification:', layout=UPD_INFO_layout)

  def get_acc_info_function():
    c.execute("select * from student where stu_id = '" + str(id) + "'")
    for acc_info_item in c:
      Info_id_textbox.value = str(acc_info_item[0])
      UPD_name_textbox.value = acc_info_item[1]
      global old_stu_username
      old_stu_username = acc_info_item[2]
      UPD_username_textbox.value = acc_info_item[2]
      UPD_password_textbox.value = acc_info_item[3]
      UPD_email_textbox.value = acc_info_item[4]
      UPD_address_textbox.value = acc_info_item[5]
      try:
        UPD_job_textbox.value = acc_info_item[6]
      except:
        UPD_job_textbox.value = ""
      try:
        UPD_qualification_textbox.value = acc_info_item[7]
      except:
        UPD_qualification_textbox.value = ""
    
  stuaccinfo1HBox = widgets.HBox([Info_id_label, Info_id_textbox])
  stuaccinfo2HBox = widgets.HBox([UPD_INFO_name_label, UPD_name_textbox])
  stuaccinfo3HBox = widgets.HBox([UPD_INFO_username_label, UPD_username_textbox])
  stuaccinfo4HBox = widgets.HBox([UPD_INFO_email_label, UPD_email_textbox])
  stuaccinfo4_1HBox = widgets.HBox([UPD_INFO_address_label, UPD_address_textbox])
  stuaccinfo5HBox = widgets.HBox([UPD_INFO_job_label, UPD_job_textbox])
  stuaccinfo6HBox = widgets.HBox([UPD_INFO_quali_label, UPD_qualification_textbox])

  UPD_info_button = widgets.Button(description="Update")

# define update info button (enter) and allow user to choose which user info to update
  def on_UPD_info_button_clicked(b):
    all_username_list = []
    c.execute("select stu_username from student")
    for row in c:
      all_username_list.append(row[0])

    update_tab_output = widgets.Output()
    acc_info_err_msg = """Please follow the rules: """
    if len(UPD_name_textbox.value) == 0:
      acc_info_err_msg += """name cannot be empty
      , """
    elif len(UPD_name_textbox.value) > 20:
      acc_info_err_msg += """input new user full name of less than 20 characters
      , """
    
    if UPD_username_textbox.value != old_stu_username and UPD_username_textbox.value in all_username_list:
      acc_info_err_msg += """username is occupied by another user
      , """
    elif len(UPD_username_textbox.value) == 0:
      acc_info_err_msg += """username cannot be empty
      , """
    elif len(UPD_username_textbox.value) > 20:
      acc_info_err_msg += """input new username of less than 20 characters
      , """
    if len(UPD_password_textbox.value) == 0:
      acc_info_err_msg += """password cannot be empty
      , """
    elif len(UPD_password_textbox.value) > 20:
      acc_info_err_msg += """input new password of less than 20 characters
      , """
    if len(UPD_email_textbox.value) == 0:
      acc_info_err_msg += """email cannot be empty
      , """
    elif len(UPD_email_textbox.value) > 20:
      acc_info_err_msg += """input new email of less than 20 characters
      , """
    if len(UPD_address_textbox.value) == 0:
      acc_info_err_msg += """address cannot be empty
      , """
    elif len(UPD_address_textbox.value) > 50:
      acc_info_err_msg += """input new address of less than 50 characters
      , """
    if len(UPD_job_textbox.value) > 50:
      acc_info_err_msg += """input new job of less than 50 characters
      , """
    if len(UPD_qualification_textbox.value) > 50:
      acc_info_err_msg += """input new qualification of less than 50 characters
      , """
    if "@" not in UPD_email_textbox.value or "." not in UPD_email_textbox.value:
      acc_info_err_msg += """input email with '@gmail.com' or '@yahoo.com.hk'
      , """   
    
    if len(acc_info_err_msg) > len("""Please follow the rules: """):
      update_status_label.value = acc_info_err_msg[0:(len(acc_info_err_msg)-2)]
# update system data
    elif len(acc_info_err_msg) == len("""Please follow the rules: """):
      c.execute(
          """
        UPDATE STUDENT 
        SET stu_name = '{}',
        stu_username = '{}',
        stu_pwd = '{}',
        stu_email = '{}',
        stu_address = '{}',
        stu_job = '{}',
        stu_qualification = '{}'
        WHERE stu_id = {}
        """.format(UPD_name_textbox.value, UPD_username_textbox.value, UPD_password_textbox.value, UPD_email_textbox.value, UPD_address_textbox.value, UPD_job_textbox.value, UPD_qualification_textbox.value, Info_id_textbox.value))
      c.execute("commit")
      update_status_label.value = "update success"
      with update_tab_output:
        textboxes_list = [UPD_name_textbox, UPD_username_textbox, UPD_password_textbox, UPD_email_textbox, UPD_address_textbox, UPD_job_textbox, UPD_qualification_textbox, Info_id_textbox]
        for all_item in textboxes_list:
          textboxes_list[all_item].clear_output()
        display(widgets.VBox([update_status_label]))
    
    time.sleep(1)
    showPage("Student Home")
  UPD_info_button.on_click(on_UPD_info_button_clicked)

  display(widgets.VBox([stuaccinfo1HBox, stuaccinfo2HBox, stuaccinfo3HBox, stuaccinfo4HBox, stuaccinfo4_1HBox, stuaccinfo5HBox, stuaccinfo6HBox, UPD_info_button, update_status_label]))

################ Account Information Tab - End ################

###########################################################
##################### Course Catalog Page (Cathy)#####################
###########################################################
with tb_stu_course.output_to('Course Catalog'):
  # create 1 label to put search error msg
  course_not_found_msg_label = Label('')

  # create header w/ combobox and button for student to search course and switch to "Bookmarks" page
  search_course_combobox = widgets.Combobox(placeholder='Course Code Search...', ensure_option=False, layout={'width':'250px'}, value=available_course_code('course')[0], options= available_course_code('course'))
  search_course_btn = widgets.Button(description='Search')
  course_detail_btn = widgets.Button(description='Show Details')
  bookmarks_btn = widgets.Button(description='Bookmarks')

  # disable bookmarks_btn if there's no record inside
  c.execute("SELECT * FROM BOOKMARK WHERE STU_ID = '"+ str(id) +"'")
  if not c.fetchall():
    bookmarks_btn.disabled = True
  
  # create a output area to store all HBoxes of retrieved data row (define outside button to enable clearing output area before retrieve another data)
  course_catalog_output = widgets.Output()
  

  # def on_course_detail_btn_click(b):
  # create 14 lists to store each column of course data, 1 list for a row of course, 1 nested list for these lists
  all_course_id = []
  all_course_code = []
  all_course_name = []
  all_course_credit = []
  all_course_section = []
  all_course_date_time = []
  all_course_room = []
  all_course_ins = []
  all_course_quota = []
  all_course_enrol = []
  all_course_avail = []
  all_course_wait = []
  all_course_description = []
  all_course_fee = []
  all_course_HBoxes = []
  all_course_info_list = [all_course_id, all_course_code, all_course_name, all_course_credit, all_course_section, all_course_date_time, all_course_room, 
                          all_course_ins, all_course_quota, all_course_enrol, all_course_avail, all_course_wait, all_course_description, all_course_fee, all_course_HBoxes]

  def search_course():
    # reset all variable (output area, msg label, lists)
    course_catalog_output.clear_output()
    course_not_found_msg_label.value = ''
    for ls in all_course_info_list:
      ls.clear()

    # create 1 label 1 list to store add_bookmark_btn and added_bm_msg_label of each row
    all_save_btn = []
    all_added_bm_msg_label = []

    ##------ define inside the search_course, as the varible is local variable
    def on_add_bookmark_btn_clicked(b):
      # find the btn's index and refer it to find the course_id
      for i in range(len(all_save_btn)):
        if all_save_btn[i] == b:
          course_id = all_course_id[i]
          row_added_bm_msg_label = all_added_bm_msg_label[i]
          break
      
      # find the max(bm_id) to set auto-increased
      bm_id = 0
      c.execute("SELECT max(BM_ID) FROM BOOKMARK")
      for row in c:
        if (type(row[0]) is int) and (row[0] != 0):
          bm_id = row[0]

      # check if this student had added this course into db already
      c.execute("SELECT COURSE_ID FROM BOOKMARK WHERE STU_ID='"+str(id)+"'")
      for row in c:
        if row[0] == course_id:
          row_added_bm_msg_label.value = 'It had been added to Bookmarks'
          return
      c.execute("INSERT INTO BOOKMARK VALUES ('{}', '{}', '{}')".format(bm_id+1, id, course_id))
      c.execute("commit")
      row_added_bm_msg_label.value = 'Action Succeed'
      bookmarks_btn.disabled = False
    ##------

    # retrieve data from database and append values of each column along with the "Save" button into lists
    c.execute("SELECT * FROM COURSE WHERE COURSE_CODE LIKE '%"+ search_course_combobox.value.upper()+"%' ORDER BY COURSE_CODE, COURSE_SECTION")
    for row in c:
      all_course_id.append(row[0])
      all_course_code.append(row[2])
      all_course_name.append(row[3])
      all_course_credit.append(row[4])
      all_course_section.append(Text(row[5], disabled=True, layout={'width': '60px'}))
      all_course_date_time.append(Text(str(row[6])+' '+str(row[7])+' - '+str(row[8]), disabled=True))
      all_course_room.append(Text(row[9], disabled=True))
      all_course_ins.append(Text(row[10], disabled=True))
      all_course_quota.append(Text(str(row[11]), disabled=True, layout={'width': '50px'}))
      all_course_enrol.append(Text(str(row[12]), disabled=True, layout={'width': '50px'}))
      all_course_avail.append(Text(str(row[13]), disabled=True, layout={'width': '50px'}))
      all_course_wait.append(Text(str(row[14]), disabled=True, layout={'width': '50px'}))
      all_course_description.append(Text(row[15], disabled=True))
      all_course_fee.append(Text(str(row[16]), disabled=True, layout={'width': '50px'}))
      
      add_bookmark_btn = widgets.Button(description='Save', layout={'width': '80px'})
      add_bookmark_btn.on_click(on_add_bookmark_btn_clicked)
      all_save_btn.append(add_bookmark_btn)

      added_bm_msg_label = Label('', layout={'height': '40px'})
      all_added_bm_msg_label.append(added_bm_msg_label)
    
    # reset the search combobox, set the error msg if nothing is found
    search_course_combobox.value = ''
    if all_course_code == []:
      course_not_found_msg_label.value = 'Course code not found. Please try again.'
      return

    # design the layout of retrieved courses row by row
    header = widgets.HBox([Text('Section', layout={'width': '60px'}, disabled=True), Text('Date & Time', disabled=True), Text('Room', disabled=True), Text('Instructor', disabled=True), 
        Text('Quota', layout={'width': '50px'}, disabled=True), Text('Enrol', layout={'width': '50px'}, disabled=True), Text('Avail', layout={'width': '50px'}, disabled=True), Text('Wait', layout={'width': '50px'}, disabled=True), 
        Text('Description', disabled=True), Text('Fee', layout={'width': '50px'}, disabled=True), widgets.Button(layout={'width': '80px'}, disabled=True)])
    for i in range(len(all_course_code)):
      if all_course_credit[i] <=1:
        course_title = HTML('<h3>'+ all_course_code[i] +' - '+ all_course_name[i] +' ('+ str(all_course_credit[i])+ ' unit) <h3>', layout={'height': '40px'})
      else:
        course_title = HTML('<h3>'+ all_course_code[i] +' - '+ all_course_name[i] +' ('+ str(all_course_credit[i])+ ' units) <h3>', layout={'height': '40px'})
            
      course_title_w_msg_label = widgets.HBox([course_title, space, all_added_bm_msg_label[i]])
      row_course_HBox = widgets.HBox([all_course_section[i], all_course_date_time[i], all_course_room[i], all_course_ins[i], all_course_quota[i], 
                                      all_course_enrol[i], all_course_avail[i], all_course_wait[i], all_course_description[i], all_course_fee[i], 
                                      all_save_btn[i]])
      
      # decide whether print the title
      if (i == 0) or (all_course_code[i] != all_course_code[i-1]):
          course_briefing = widgets.VBox([course_title_w_msg_label, header, row_course_HBox])    
      else:
        course_briefing = row_course_HBox
      all_course_HBoxes.append(course_briefing)
    
    # display all data in the output area row by row
    with course_catalog_output:
      display(widgets.VBox([row for row in all_course_HBoxes]))
  
  def on_search_course_btn_clicked(b):
    search_course()
  search_course_btn.on_click(on_search_course_btn_clicked)

  def on_bookmarks_btn_clicked(b):
    course_catalog_output.clear_output()
    search_course_combobox.value = ''
    with tb_stu_course.output_to('Bookmarks'):
      display()
  bookmarks_btn.on_click(on_bookmarks_btn_clicked)

  search_block = widgets.HBox([search_course_combobox, search_course_btn, space, bookmarks_btn])
  msg_block = widgets.HBox([course_not_found_msg_label])
  display(widgets.VBox([search_block, msg_block, spacer, course_catalog_output]))
  search_course()
#################### Course Catalog Page - End#####################

###########################################################
##################### Bookmarks Page (Cathy)#####################
###########################################################
with tb_stu_course.output_to('Bookmarks'):
  # create 2 label to store 'error'/'action succeed' msg
  bookmark_not_found_msg_label = Label('')
  bookmark_deleted_msg_label = Label('')

  # create 2 buttons and a combobox to search bookmarked courses and go back to course catalog
  search_bookmark_combobox = widgets.Combobox(placeholder='Course Code Search...', ensure_option=False, layout={'width':'250px'}, options= available_course_code('course'))
  search_bookmark_btn = widgets.Button(description='Search')
  search_all_bookmark_btn = widgets.Button(description='Search All')
  course_catalog_btn = widgets.Button(description='Course Catalog')
  
  # create output area
  bookmark_output = widgets.Output()

  def bm_to_course_catalog():
    search_course_combobox.value = available_course_code('course')[0]
    search_course()
    with tb_stu_course.output_to('Course Catalog'):
      display()
    search_bookmark_combobox.value == ''
    bookmark_not_found_msg_label.value = ''

  def search_bookmark():
    # reset variable (output area, msg labels, lists)
    bookmark_output.clear_output()
    bookmark_not_found_msg_label.value = ''
    bookmark_deleted_msg_label = ''
    for ls in all_course_info_list:
      ls.clear()

    # create 1 label and 1 list to store remove_bm_btn and bookmark_removed_msg_label of each row
    all_remove_bm_btn = []
    all_bookmark_removed_msg_label = []

    ##------
    def on_remove_bm_btn_click(b):
      # find which row of course the button is
      for i in range(len(all_remove_bm_btn)):
        if all_remove_bm_btn[i] == b:
          course_id = all_course_id[i]
          row_bookmark_deleted_msg_label = all_bookmark_removed_msg_label[i]
          break

      # show error msg in case users double click in a short time
      c.execute("SELECT * FROM bookmark WHERE course_id = '"+ str(course_id) + "' AND stu_id = '" + str(id) + "'")
      if not c.fetchall():
        row_bookmark_deleted_msg_label.value = 'The bookmark had been removed'
        return
      
      # delete record from db
      c.execute("DELETE FROM bookmark WHERE (course_id = '" + str(course_id) + "' AND stu_id = '" + str(id) + "')")
      c.execute('commit')
      
      # refresh the tab and show msg
      row_bookmark_deleted_msg_label.value = 'Action Succeed'
      search_bookmark_combobox.value = ''
      time.sleep(1)
      search_bookmark()
    ##-----

    # retrieve course data and append them into lists
    if search_bookmark_combobox.value == '':
      c.execute("SELECT * FROM course c INNER JOIN bookmark b ON c.course_id = b.course_id WHERE b.stu_id = '" + str(id) + "'")
    else:
      c.execute("""SELECT * FROM (SELECT * FROM course c INNER JOIN bookmark b ON c.course_id = b.course_id WHERE b.stu_id = '""" + str(id) + """') 
                   WHERE course_code LIKE '%"""+ search_bookmark_combobox.value.upper() + """%'""")
    for row in c:
      all_course_id.append(row[0])
      all_course_code.append(row[2])
      all_course_name.append(row[3])
      all_course_credit.append(row[4])
      all_course_section.append(Text(row[5], disabled=True, layout={'width': '60px'}))
      all_course_date_time.append(Text(str(row[6])+' '+str(row[7])+' - '+str(row[8]), disabled=True))
      all_course_room.append(Text(row[9], disabled=True))
      all_course_ins.append(Text(row[10], disabled=True))
      all_course_quota.append(Text(str(row[11]), disabled=True, layout={'width': '50px'}))
      all_course_enrol.append(Text(str(row[12]), disabled=True, layout={'width': '50px'}))
      all_course_avail.append(Text(str(row[13]), disabled=True, layout={'width': '50px'}))
      all_course_wait.append(Text(str(row[14]), disabled=True, layout={'width': '50px'}))
      all_course_description.append(Text(row[15], disabled=True))
      all_course_fee.append(Text(str(row[16]), disabled=True, layout={'width': '50px'}))

      remove_bm_btn = widgets.Button(description='Remove', layout={'width':'100px'})
      remove_bm_btn.on_click(on_remove_bm_btn_click)
      all_remove_bm_btn.append(remove_bm_btn)
      
      bookmark_deleted_msg_label = Label('')
      all_bookmark_removed_msg_label.append(bookmark_deleted_msg_label)

    # show error msg if nothing is found, and reset the search combobox
    if (search_bookmark_combobox.value == '') and (all_course_code == []):
      bookmark_not_found_msg_label.value = 'Bookmarks is empty. Please check out Course Catalog first.'
      time.sleep(1)
      bm_to_course_catalog()
      return
    elif all_course_code == []:
      bookmark_not_found_msg_label.value = 'Course code not found.'
      search_bookmark_combobox.value == ''
      return

    # design the layout of retrieved courses row by row
    header = widgets.HBox([Text('Section', layout={'width': '60px'}, disabled=True), Text('Date & Time', disabled=True), Text('Room', disabled=True), Text('Instructor', disabled=True), 
        Text('Quota', layout={'width': '50px'}, disabled=True), Text('Enrol', layout={'width': '50px'}, disabled=True), Text('Avail', layout={'width': '50px'}, disabled=True), Text('Wait', layout={'width': '50px'}, disabled=True), 
        Text('Description', disabled=True), Text('Fee', layout={'width': '50px'}, disabled=True), widgets.Button(layout={'width': '100px'}, disabled=True)])
    for i in range(len(all_course_code)):
      if all_course_credit[i] <=1:
        course_title = HTML('<h3>'+ all_course_code[i] +' - '+ all_course_name[i] +' ('+ str(all_course_credit[i])+ ' unit) <h3>', layout={'height': '40px'})
      else:
        course_title = HTML('<h3>'+ all_course_code[i] +' - '+ all_course_name[i] +' ('+ str(all_course_credit[i])+ ' units) <h3>', layout={'height': '40px'})
            
      course_title_w_msg_label = widgets.HBox([course_title, space, all_bookmark_removed_msg_label[i]])
      row_course_HBox = widgets.HBox([all_course_section[i], all_course_date_time[i], all_course_room[i], all_course_ins[i], all_course_quota[i], 
                                      all_course_enrol[i], all_course_avail[i], all_course_wait[i], all_course_description[i], all_course_fee[i], 
                                      all_remove_bm_btn[i]])
      
      # decide whether print the title
      if (len(all_course_code) == 1) or (all_course_code[i] != all_course_code[i-1]):
          course_briefing = widgets.VBox([course_title_w_msg_label, header, row_course_HBox])    
      else:
        course_briefing = row_course_HBox
      all_course_HBoxes.append(course_briefing)
    
    # display all data in the output area row by row
    with bookmark_output:
      display(widgets.VBox([row for row in all_course_HBoxes]))

  def on_search_bookmark_btn_click(b):
    bookmark_output.clear_output()
    search_bookmark()
  search_bookmark_btn.on_click(on_search_bookmark_btn_click)

  def on_search_all_bookmark_btn_click(b):
    bookmark_output.clear_output()
    search_bookmark_combobox.value = ''
    search_bookmark()
  search_all_bookmark_btn.on_click(on_search_all_bookmark_btn_click)

  def on_course_catalog_btn_click(b):
    bookmark_output.clear_output()
    bm_to_course_catalog()
  course_catalog_btn.on_click(on_course_catalog_btn_click)

  search_block = widgets.HBox([course_catalog_btn, Label(''), search_bookmark_combobox, search_bookmark_btn, search_all_bookmark_btn])
  msg_block = widgets.HBox([bookmark_not_found_msg_label])
  display(widgets.VBox([search_block, msg_block, spacer, bookmark_output]))
#################### Bookmarks Page - End#####################

###########################################################
##################### Enrollment Cart Page (Susanna)#####################
###########################################################
with tb_stu_course.output_to('Enrollment Cart'):
# create label 
  Enroll_label = Label("Enroll into your course")
  enroll_status_label = widgets.Output()
  credit_card_no_textbox = widgets.Output()
  enroll_button_output = widgets.Output()
  ret_course_time_button_output = widgets.Output()
  get_course_code_for_enrollment_cart_output= widgets.Output()
  get_course_code_from_bookmark_output= widgets.Output()
# retrieve all course codes
  course_code = ["course code"]
  current_date = str(date.today().strftime('%d-%b-%y')).upper()
  
  ret_course_time_button = widgets.Button(description="See timeslot")
  
  enroll_btn = widgets.Button(description="Enroll")
  enroll2_button = widgets.Button(description="Enroll")
  confirm_btn = widgets.Button(description="Confirm")
  insert_button = widgets.Button(description="Insert")
  enrollment_cart_course_date = []
  enrollment_cart_course_starttime = []
  enrollment_cart_course_startdate = []
  enrollment_cart_course_enddate = []
  enrollment_cart_course_endtime = []
  
  enrollment_cart_course_id = []
  enrollment_cart_course_code = []
  enrollment_cart_course_info = []
  enrollment_cart_course_timeslot = []
  global enrollment_cart_Course_Desc, enrollment_cart_Course_Quota, enrollment_cart_Course_Instructor, enrollment_cart_Course_Enrollment_Fee, enrollment_cart_course_name, enrollment_cart_course_id__of_choice, course_code_choice_list
  enrollment_cart_Course_Desc = ""
  enrollment_cart_Course_Quota = ""
  enrollment_cart_Course_Instructor = ""
  enrollment_cart_Course_Enrollment_Fee = ""
  enrollment_cart_course_name = ""
  enrollment_cart_course_id__of_choice = 0
  # define variables to store values
  next_enr_id = 0
  S_course_id = 0
  S_course_enroll = 0
  S_course_quota = 0
  S_course_wait = 0
  S_course_avail = 0
# create dropdown list and buttons and lists
  
  course_code_choice_list = widgets.Dropdown(options = enrollment_course_code(id), description='From all courses:', disabled=False)
  bookmark_course_code_choice_list = widgets.Dropdown(description='From bookmarked courses:', disabled=False)
  
  ret_bookmark_course_code_btn = widgets.Button(description='Retrieve Bookmark')
  def on_ret_bookmark_course_code_btn_click(b):
    bookmark_course_code_choice_list.options = bookmark_course_code(id)
  ret_bookmark_course_code_btn.on_click(on_ret_bookmark_course_code_btn_click)

  enroll_label_ret_btn = widgets.VBox([Enroll_label, ret_bookmark_course_code_btn])
  Enroll_row1VBox = widgets.VBox([course_code_choice_list, bookmark_course_code_choice_list])
  Enroll_row2VBox = widgets.VBox([ret_course_time_button, enroll2_button])
  row1_HBox = widgets.HBox([enroll_label_ret_btn, Enroll_row1VBox, Enroll_row2VBox])



  def on_enroll2_button_clicked(self):
    enroll_button_output.clear_output()
    global course_timeslot_index, enrollment_cart_course_id_of_choice, enrollment_cart_Course_Desc, enrollment_cart_Course_Quota, enrollment_cart_Course_Instructor, enrollment_cart_Course_Enrollment_Fee, enrollment_cart_course_name
    bm_index = bookmark_course_code(id).index(bookmark_course_code_choice_list.value)
    
    c.execute("""
    SELECT course_name, course_desc, course_avail, course_instructor, course_enrollment_fee, course_id
    FROM COURSE 
    WHERE course_code ='""" + bookmark_course_code_choice_list.value + "' and course_id ="
    + str(bookmark_course_code_1(id)[bm_index]))
    
    for i in c:
      if i[5] != None:
        enrollment_cart_course_id_of_choice = str(i[5])
      elif i[5] == None:
        enrollment_cart_course_id_of_choice = "Nil"

      if i[0] != None:
        enrollment_cart_course_name = str(i[0])
      elif i[0] == None:
        enrollment_cart_course_name = "Nil"
      if i[1] != None:
        enrollment_cart_Course_Desc = str(i[1])
      elif i[1] == None:
        enrollment_cart_Course_Desc = "Nil"
      if i[2] != None:
        enrollment_cart_Course_Quota = str(i[2])
      elif i[2] == None:
        enrollment_cart_course_Quota = "Nil"
      if i[3] != None:
        enrollment_cart_Course_Instructor = str(i[3])
      elif i[3] == None:
        enrollment_cart_course_Instructor = "Nil"
      if i[4] != None:
        enrollment_cart_Course_Enrollment_Fee = str(i[4])
      elif i[4] == None:
        enrollment_cart_course_Enrollment_Fee = "Nil"
      

      Enroll_row12VBox = widgets.HBox([Label("Course ID:"), Label(str(enrollment_cart_course_id_of_choice))])
      Enroll_row5VBox = widgets.HBox([Label("Course Name:"), Label(enrollment_cart_course_name)])
      Enroll_row6VBox = widgets.HBox([Label("Course Description:"), Label(enrollment_cart_Course_Desc)])
      Enroll_row8VBox = widgets.HBox([Label("Instructor:"), Label(enrollment_cart_Course_Instructor)])
      Enroll_row9VBox = widgets.HBox([Label("Enrollment Fee:"), Label(str(enrollment_cart_Course_Enrollment_Fee))])
      Enroll_row10VBox = widgets.HBox([Label("Available quota:"), Label(str(enrollment_cart_Course_Quota))])
    
    Enroll_row11VBox = widgets.HBox([confirm_btn])
    with enroll_button_output:
      display(widgets.VBox([Enroll_row12VBox, Enroll_row5VBox, Enroll_row6VBox, Enroll_row8VBox, Enroll_row9VBox, Enroll_row10VBox, Enroll_row11VBox]))

  enroll2_button.on_click(on_enroll2_button_clicked)

# define button to retrieve course time 
  def on_ret_course_time_button_clicked(self):
    ret_course_time_button_output.clear_output()
    c.execute("""SELECT course_day, course_starttime, course_endtime, 
    TO_CHAR(course_startdate,'DD-MM-YY'), TO_CHAR(course_enddate,'DD-MM-YY'), course_id 
    from COURSE WHERE course_code ='""" 
    + str(course_code_choice_list.value)
    + "' and course_enddate > TO_DATE(SYSDATE, 'DD-MM-YY')")
    
    for row in c:
      if row[0] != None:
        enrollment_cart_course_date.append(str(row[0]))
      else:
        enrollment_cart_course_date.append("Nil")
      if row[1] != None:
        enrollment_cart_course_starttime.append(str(row[1]))
      else:
        enrollment_cart_course_starttime.append("Nil")
      if row[2] != None:
        enrollment_cart_course_endtime.append(str(row[2]))
      else:
        enrollment_cart_course_endtime.append("Nil")
      if row[3] != None:
        enrollment_cart_course_startdate.append(str(row[3]))
      else:
        enrollment_cart_course_startdate.append("Nil")
      if row[4] != None:
        enrollment_cart_course_enddate.append(str(row[4]))
      else:
        enrollment_cart_course_enddate.append("Nil")
      if row[5] != None:
        enrollment_cart_course_id.append(str(row[5]))
      else:
        enrollment_cart_course_id.append("Nil")

      for item in range(len(enrollment_cart_course_id)):
        course_timeslot_item = enrollment_cart_course_date[item] + " " + enrollment_cart_course_startdate[item] + "-" + enrollment_cart_course_enddate[item] + " " + enrollment_cart_course_starttime[item] + "-" + enrollment_cart_course_endtime[item]
        global enrollment_cart_course_timeslot
        enrollment_cart_course_timeslot.append(course_timeslot_item)
    global course_timeslot_choice_list    
    course_timeslot_choice_list = widgets.Dropdown(options = enrollment_cart_course_timeslot, description='Timeslot:', disabled=False, layout={'width': '500px'})
    Enroll_row3VBox = widgets.VBox([course_timeslot_choice_list])
    Enroll_row4VBox = widgets.VBox([enroll_btn])
    with ret_course_time_button_output:
      HBox = (widgets.HBox([Enroll_row3VBox, Enroll_row4VBox]))
      display(widgets.VBox([spacer, HBox]))

  ret_course_time_button.on_click(on_ret_course_time_button_clicked)

# define button to retrieve course detail
  def on_enroll_btn_clicked(self):
    enroll_button_output.clear_output()
    global course_timeslot_index, enrollment_cart_course_id_of_choice, enrollment_cart_Course_Desc, enrollment_cart_Course_Quota, enrollment_cart_Course_Instructor, enrollment_cart_Course_Enrollment_Fee, enrollment_cart_course_name
    course_timeslot_index = enrollment_cart_course_timeslot.index(course_timeslot_choice_list.value)
    
    c.execute("""
    SELECT course_name, course_desc, course_avail, course_instructor, course_enrollment_fee, course_id
    FROM COURSE 
    WHERE course_code ='""" + course_code_choice_list.value + "' and course_id ="
    + str(enrollment_cart_course_id[course_timeslot_index]))
    
    for i in c:
        if i[5] != None:
          enrollment_cart_course_id_of_choice = (i[5])
        elif i[5] == None:
          enrollment_cart_course_id_of_choice = "Nil"

        if i[0] != None:
          enrollment_cart_course_name = (i[0])
        elif i[0] == None:
          enrollment_cart_course_name = "Nil"
        if i[1] != None:
          enrollment_cart_Course_Desc = (i[1])
        elif i[1] == None:
          enrollment_cart_Course_Desc = "Nil"
        if i[2] != None:
          enrollment_cart_Course_Quota = (i[2])
        elif i[2] == None:
          enrollment_cart_course_Quota = "Nil"
        if i[3] != None:
          enrollment_cart_Course_Instructor = (i[3])
        elif i[3] == None:
          enrollment_cart_course_Instructor = "Nil"
        if i[4] != None:
          enrollment_cart_Course_Enrollment_Fee = (i[4])
        elif i[4] == None:
          enrollment_cart_course_Enrollment_Fee = "Nil"


    Enroll_row12VBox = widgets.HBox([Label("Course ID:"), Label(str(enrollment_cart_course_id_of_choice))])
    Enroll_row5VBox = widgets.HBox([Label("Course Name:"), Label(enrollment_cart_course_name)])
    Enroll_row6VBox = widgets.HBox([Label("Course Description:"), Label(enrollment_cart_Course_Desc)])
    Enroll_row8VBox = widgets.HBox([Label("Instructor:"), Label(enrollment_cart_Course_Instructor)])
    Enroll_row9VBox = widgets.HBox([Label("Enrollment Fee:"), Label(str(enrollment_cart_Course_Enrollment_Fee))])
    Enroll_row10VBox = widgets.HBox([Label("Available quota:"), Label(str(enrollment_cart_Course_Quota))])
    
    Enroll_row11VBox = widgets.HBox([confirm_btn])
    with enroll_button_output:
      display(widgets.VBox([spacer, Enroll_row12VBox, Enroll_row5VBox, Enroll_row6VBox, Enroll_row8VBox, Enroll_row9VBox, Enroll_row10VBox, Enroll_row11VBox]))
  enroll_btn.on_click(on_enroll_btn_clicked)

# define button to insert enrollment record
  def on_confirm_btn_clicked(self):
    global credit, CVV, expiry_date
    credit = Text(description='Credit card number:', value='', placeholder='credit card number', disabled=False)
    CVV = Text(description='CVV number:', value='', placeholder='CVV number', disabled=False)
    expiry_date = Text(description='Expiry date:', value='', placeholder='expiry date', disabled=False)
    with credit_card_no_textbox:
      display(widgets.VBox([credit, CVV, expiry_date, insert_button]))
    #showPage("Enrollment Cart")
  confirm_btn.on_click(on_confirm_btn_clicked)
        

  def on_insert_button_clicked(self):
    enroll_status_message = Text(value='', disabled=False)
    
    c.execute("SELECT MAX(enr_id) FROM ENROLLMENT")
    for item in c:
      next_enr_id = item[0]

    c.execute("SELECT course_enrol, course_quota, course_wait, course_avail FROM COURSE WHERE course_id =" + str(enrollment_cart_course_id_of_choice))
    for i in c:
      S_course_enroll = i[0]
      S_course_quota = i[1]
      S_course_wait = i[2]
      S_course_avail = i[3]
    
    if S_course_quota <= S_course_enroll:
      wait_no = S_course_wait + 1
      enroll_status_message.value = "You are in the waitlist. Your waitlist number is " + str(wait_no)
      enroll_status = "waitlist"
      enroll_attendance = "waiting"
    elif S_course_quota > S_course_enroll:
      enroll_status = "enrolled"
      enroll_attendance = "no attendance yet"
      enroll_status_message.value = "you are succesfully enrolled"


    if credit.value != "":
      c.execute("INSERT INTO ENROLLMENT (enr_id, stu_id, course_id, enr_status, enr_attendence, enr_date) VALUES ({}, {}, {}, '{}', '{}', sysdate)".format(str(next_enr_id+1), str(id), str(enrollment_cart_course_id_of_choice), enroll_status, enroll_attendance))
      if enroll_status == "enrolled":
        c.execute("update course set course_avail=" + str(S_course_avail-1) + ", course_enrol=" + str(S_course_enroll+1) + "where course_id = " + str(enrollment_cart_course_id_of_choice))
        # +  
        # Question: no need to update course_avail manually?
      elif enroll_status == "waitlist":
        c.execute("update course set course_wait=" + str(wait_no))
      c.execute("commit")

    elif credit.value == "":
      enroll_status_message.value = "please input valid credit card number"
    with enroll_status_label:
      display(enroll_status_message)
      #get_course_code_for_enrollment_cart()
      #get_course_code_from_bookmark()
      
  insert_button.on_click(on_insert_button_clicked)

  display(widgets.VBox([row1_HBox, ret_course_time_button_output, enroll_button_output, credit_card_no_textbox, enroll_status_label ]))
  
#################### Enrollment Cart Page - End#####################

###########################################################
##################### View Enrollment Page (Susanna)#####################
###########################################################
with tb_stu_course.output_to('View Enrollment'):
########
# create 3 buttons
  ret_all_enr_button = widgets.Button(description="Retrieve All")
  ret_all_enr_clear_button = widgets.Button(description="Clear All")
  ret_all_enr_back_button = widgets.Button(description="Back")
  ret_all_enr_out = widgets.Output()
# create label and lists
  ViewEnrollment_label = Label("Please find your enrolled courses")


  ret_all_enr_id_textboxes = []
  ret_all_enr_status_textboxes = []
  ret_all_course_id_textboxes = []
  ret_all_course_name_textboxes = []
  ret_all_course_date_textboxes = []
  ret_all_course_starttime_textboxes = []
  ret_all_course_startdate_textboxes = []
  ret_all_course_enddate_textboxes = [] 
  ret_all_course_endtime_textboxes = [] 
  ret_all_course_timeslot_textboxes = [] 
  ret_all_course_room_textboxes = []
  ret_all_course_instructor_textboxes = []
  ret_all_course_enroll_textboxes = []
  ret_all_course_wait_textboxes = []
  ret_all_course_desc_textboxes = []
  ret_all_course_enrollment_fee_textboxes = []
  ret_all_course_quota_textboxes = []
  ret_all_course_credit_textboxes = []
  ret_all_course_avail_textboxes = []
  ret_all_course_section_textboxes = []

  ret_all_enr_HBoxes = []
  view_enroll_result_label = Label("")
 

  def on_ret_all_enr_clear_button_clicked(b):
    box_list = [ret_all_enr_id_textboxes, ret_all_enr_status_textboxes, ret_all_course_id_textboxes, ret_all_course_name_textboxes, 
                ret_all_course_timeslot_textboxes, ret_all_course_room_textboxes, ret_all_course_instructor_textboxes, ret_all_course_enroll_textboxes, 
                ret_all_course_wait_textboxes, ret_all_course_desc_textboxes, ret_all_course_enrollment_fee_textboxes, ret_all_course_starttime_textboxes, 
                ret_all_course_startdate_textboxes, ret_all_course_enddate_textboxes, ret_all_course_endtime_textboxes, ret_all_course_date_textboxes,
                ret_all_course_credit_textboxes, ret_all_course_quota_textboxes,  ret_all_course_avail_textboxes, ret_all_course_section_textboxes]
    for item in range(len(box_list)):
      box_list[item].clear()
    ret_all_enr_HBoxes.clear()
    ret_all_enr_out.clear_output()
  ret_all_enr_clear_button.on_click(on_ret_all_enr_clear_button_clicked)


  def on_ret_all_enr_button_clicked(b):
    on_ret_all_enr_clear_button_clicked(b)

    c.execute("""SELECT Enr_ID, Enr_Status, 
    c.course_id, c.course_name, c.course_day, c.course_starttime, c.course_endtime, c.course_room, 
    c.course_instructor, c.course_enrol, c.course_wait, c.course_desc, c.course_enrollment_fee, 
    TO_CHAR(c.course_startdate, 'DD-MON-YY'), TO_CHAR(c.course_enddate, 'DD-MON-YY'), c.course_quota, c.course_credit, course_avail, course_section
    from Enrollment e 
    INNER JOIN COURSE c
    ON e.course_id = c.course_id
    WHERE e.stu_id = """ + str(id))

    for row in c:
      if row[4] != None:
        ret_all_course_date_textboxes.append(row[4])
      elif row[4] == None:
        ret_all_course_date_textboxes.append("Nil")
      if row[5] != None:
        ret_all_course_starttime_textboxes.append(row[5])
      elif row[5] == None:
        ret_all_course_starttime_textboxes.append("Nil")
      if row[6] != None:
        ret_all_course_endtime_textboxes.append(row[6])
      elif row[6] == None:
        ret_all_course_endtime_textboxes.append("Nil")
      if row[13] != None:
        ret_all_course_startdate_textboxes.append(row[13])
      elif row[13] == None:
        ret_all_course_startdate_textboxes.append("Nil")
      if row[14] != None:
        ret_all_course_enddate_textboxes.append(row[14])
      elif row[14] == None:
        ret_all_course_enddate_textboxes.append("Nil")

      for i in range(len(ret_all_course_starttime_textboxes)):
        if row[4] == None and row[5] == None and row[6] == None and row[13] == None and row[14] == None:
          ret_all_course_timeslot_textboxes.append(Text(value="Nil", disabled=True))
        else:
          timeslot_txt = ret_all_course_date_textboxes[i] + " " + ret_all_course_startdate_textboxes[i] + "-" + ret_all_course_enddate_textboxes[i] + " " + ret_all_course_starttime_textboxes[i] + "-" + ret_all_course_endtime_textboxes[i]
          ret_all_course_timeslot_textboxes.append(Text(value=str(timeslot_txt), disabled=True))
      
      ret_all_enr_id_textboxes.append(Text(value=str(row[0]), disabled=True))
      
      if row[1] == 'waitlist':
        ret_all_enr_status_textboxes.append(Text(value=str(row[1]) + ": " + str(row[10]), disabled=True)) 
      else:
        ret_all_enr_status_textboxes.append(Text(value=str(row[1]), disabled=True))
      
    
      ret_all_course_id_textboxes.append(Text(value=str(row[2]), disabled=True))
      ret_all_course_name_textboxes.append(Text(value=str(row[3]), disabled=True))
      ret_all_course_credit_textboxes.append(Text(value=str(row[16]), disabled=True))

      if row[7] != None:
        ret_all_course_room_textboxes.append(Text(value=str(row[7]), disabled=True))
      elif row[7] == None:
        ret_all_course_room_textboxes.append(Text(value="Nil", disabled=True))
      if row[8] != None:
        ret_all_course_instructor_textboxes.append(Text(value=str(row[8]), disabled=True))
      elif row[8] == None:
        ret_all_course_instructor_textboxes.append(Text(value="Nil", disabled=True))
      if row[9] != None:
        ret_all_course_enroll_textboxes.append(Text(value=str(row[9]), disabled=True))
      elif row[9] == None:
        ret_all_course_enroll_textboxes.append(Text(value="Nil", disabled=True))
      if row[10] != None:
        ret_all_course_wait_textboxes.append(Text(value=str(row[10]), disabled=True))
      elif row[10] == None:
        ret_all_course_wait_textboxes.append(Text(value="Nil", disabled=True))
      if row[11] != None:
        ret_all_course_desc_textboxes.append(Text(value=str(row[11]), disabled=True))
      elif row[11] == None:
        ret_all_course_desc_textboxes.append(Text(value="Nil", disabled=True))
      if row[12] != None:
        ret_all_course_enrollment_fee_textboxes.append(Text(value=str(row[12]), disabled=True))
      elif row[12] == None:
        ret_all_course_enrollment_fee_textboxes.append(Text(value="Nil", disabled=True))
      if row[15] != None:
        ret_all_course_quota_textboxes.append(Text(value=str(row[15]), disabled=True))
      elif row[15] == None:
        ret_all_course_quota_textboxes.append(Text(value="Nil", disabled=True))
      if row[17] != None:
        ret_all_course_avail_textboxes.append(Text(value=str(row[17]), disabled=True))
      elif row[17] == None:
        ret_all_course_avail_textboxes.append(Text(value="Nil", disabled=True))      
      if row[18] != None:
        ret_all_course_section_textboxes.append(Text(value=row[18], disabled=True))
      elif row[18] == None:
        ret_all_course_section_textboxes.append(Text(value="Nil", disabled=True))            
                                
    if range(len(ret_all_enr_id_textboxes)-1) == 0:
      view_enroll_result_label.value = "No current enrollment"
    else:
      for x in range(len(ret_all_enr_id_textboxes)-1):
        ret_all_enr_HBoxes.append( widgets.HBox([ret_all_enr_id_textboxes[x], ret_all_enr_status_textboxes[x], ret_all_course_id_textboxes[x], ret_all_course_name_textboxes[x], ret_all_course_section_textboxes[x],  
        ret_all_course_credit_textboxes[x], ret_all_course_timeslot_textboxes[x], ret_all_course_room_textboxes[x], ret_all_course_instructor_textboxes[x], 
        ret_all_course_quota_textboxes[x], ret_all_course_enroll_textboxes[x], ret_all_course_avail_textboxes[i], ret_all_course_wait_textboxes[x], ret_all_course_desc_textboxes[x], ret_all_course_enrollment_fee_textboxes[x]]) )

      # ret_all_enr_HBoxes.append( widgets.HBox( ret_all_list ))

    with ret_all_enr_out:
        display(widgets.VBox([i for i in ret_all_enr_HBoxes]))
  ret_all_enr_button.on_click(on_ret_all_enr_button_clicked)



  
  def on_ret_all_enr_back_button_clicked(b):
    showPage("Course")
  ret_all_enr_back_button.on_click(on_ret_all_enr_back_button_clicked)

# display the objects
  header = widgets.HBox([Text('Enrollment ID', disabled=True), Text('Status', disabled=True), Text('Course ID', disabled=True), 
                         Text('Course Name', disabled=True), Text('Section', disabled=True), Text('Credit', disabled=True), Text('Date & Time', disabled=True), 
                         Text('Room', disabled=True), Text('Instructor', disabled=True), 
                         Text('Quota', disabled=True), Text('Enrol', disabled=True), Text('Avail', disabled=True), Text('Wait', disabled=True), 
                         Text('Description', disabled=True), Text('Fee', disabled=True)])
  ret_all_control_HBox = widgets.HBox([ret_all_enr_button, ret_all_enr_clear_button])
  ret_all_VBox = widgets.VBox([ret_all_control_HBox, spacer, header, ret_all_enr_out, ret_all_enr_back_button, view_enroll_result_label])
  display(ret_all_VBox)
#################### View Enrollment Page - End#####################


###########################################################
################ SFQ - Begin (Susanna)################
###########################################################
with tb_stu_course.output_to('SFQ'):
# Select course
  #Create COURSE label
  retrievecourse_label = Label("Select courseID and input comment here")
  sfq_result_label = Label("")
  get_course_id_for_sfq_function_output = widgets.Output()

# select course id of courses enrolled in current semester
  sfq_course_id = ["COURSE CODE"]
  course_id = ["course_id"]
  current_date = date.today()
  current_year = current_date.strftime('%y')
  if int(current_date.strftime('%m%d')) > 901:
    current_sem_start = '-SEP-'
  elif int(current_date.strftime('%m%d')) > 201 and int(current_date.strftime('%m%d')) < 901:
    current_sem_start = '-FEB-'
  def get_course_id_for_sfq_function():
    c.execute("""SELECT c.course_code, e.course_id 
    from ENROLLMENT e 
    INNER JOIN course c ON e.course_ID = c.course_ID 
    WHERE e.stu_id =""" + str(id)
    + "  and e.enr_status = 'enrolled' and e.course_id not in (select course_id from feedback where stu_id =" 
    + str(id) + ")")
  #and course_startdate LIKE TO_DATE('" + current_sem_start + "', '-MON-')
  #AND c.course_startdate = '" + current_sem_start+current_year + "'
  # assume enrollment table will be cleared every semester
    for row in c:
      sfq_course_id.append(row[0])
      course_id.append(row[1])
    global sfq_course_id_choice_list
    sfq_course_id_choice_list = widgets.Dropdown(options = sfq_course_id, value = sfq_course_id[0], description='Course:', disabled=False)
    # place textboxes
    line1VBox = widgets.HBox([retrievecourse_label, sfq_course_id_choice_list, Retrievecourseinfo_button])
    line2VBox = widgets.HBox([spacer])
    line1HBox = widgets.HBox([course_name_desc, COURSE_NAME_textbox])
    line2HBox = widgets.HBox([course_ins_desc, COURSE_INS_textbox])
    line3VBox = widgets.HBox([line1HBox, line2HBox])
    with get_course_id_for_sfq_function_output:
      display(widgets.VBox([line1VBox,  line2VBox, line3VBox, split]))

# display course info by textbox
  course_name_desc = Label("Course Name: ")
  COURSE_NAME_textbox = Text(disabled=True)
  course_ins_desc = Label("Course Instructor: ")
  COURSE_INS_textbox = Text(disabled=True)

  Retrievecourseinfo_button = widgets.Button(description="Retrieve Course")

  def on_Retrievecourseinfo_button_clicked(b):
    global index
    index = sfq_course_id.index(sfq_course_id_choice_list.value)
    c.execute("""SELECT c.COURSE_NAME, c.COURSE_INSTRUCTOR 
    FROM COURSE c
    inner join enrollment e
    on c.course_id = e.course_id
    WHERE e.COURSE_id = '""" + str(course_id[index]) + "'")
    for row in c:
      COURSE_NAME_textbox.value = row[0]
      COURSE_INS_textbox.value = row[1]


  Retrievecourseinfo_button.on_click(on_Retrievecourseinfo_button_clicked)
  
# Give feedback

# auto create feedback id
  c.execute("SELECT MAX(fb_id)+1 FROM FEEDBACK")
  for e in c:
    Feedback_ID = e[0]

# ask for comment input
  ratecourse_label = Label("Rate the course and put your comment here")
  comment_desc = Label("Overall comment: ")
  commentcourse_textbox = Text(placeholder='Comment', disabled=False, layout=widgets.Layout(height="200%"))

  display_radiobuttons_1 = widgets.RadioButtons(options=['1', '2', '3', '4', '5'], disabled=False)
  display_radiobuttons_2 = widgets.RadioButtons(options=['1', '2', '3', '4', '5'], disabled=False)
  display_radiobuttons_3 = widgets.RadioButtons(options=['1', '2', '3', '4', '5'], disabled=False)
  display_radiobuttons_4 = widgets.RadioButtons(options=['1', '2', '3', '4', '5'], disabled=False)
  display_radiobuttons_5 = widgets.RadioButtons(options=['1', '2', '3', '4', '5'], disabled=False)
  first_HBox = widgets.VBox([Label("Q1. The course materials and learning activities helped me achieve the course intended learning outcomes. *                 "), display_radiobuttons_1])
  second_HBox = widgets.VBox([Label("Q2. Assessment tasks were designed to determine the extent to which I had achieved the course intended learning outcomes. *"), display_radiobuttons_2])
  third_HBox = widgets.VBox([Label("Q3. This course was academically challenging. *                                                                             "), display_radiobuttons_3])
  fourth_HBox = widgets.VBox([Label("Q4. The instructor(s) stimulated my interest in this subject and encouraged me to think. *                                 "), display_radiobuttons_4])  
  fifth_HBox = widgets.VBox([Label("Q5. The instructor(s) was/were readily available to answer questions and support students. *                                "), display_radiobuttons_5])  
  
  input_comment_btn = widgets.Button(description="Submit Feedback")
  
  def on_input_comment_btn_clicked(b):
    # retrieve instructor id
    ratecourse = int(display_radiobuttons_1.value) + int(display_radiobuttons_2.value) + int(display_radiobuttons_3.value) + int(display_radiobuttons_4.value) + int(display_radiobuttons_5.value)

    c.execute("SELECT ins_id FROM COURSE WHERE COURSE_ID = " + str(course_id[index]))
    for item in c:
      ins_id = item[0]
    c.execute("INSERT INTO FEEDBACK VALUES ({}, {}, {}, '{}', {}, {})".format(Feedback_ID, str(id), str(course_id[index]), commentcourse_textbox.value, ratecourse, ins_id))
    c.execute("commit")
    input_comment_btn.disabled = True
    sfq_result_label.value = "Feedback is received. Thank you for your time!"

  input_comment_btn.on_click(on_input_comment_btn_clicked)

# place boxes

  ratecourse_row1VBox = widgets.VBox([ratecourse_label])
  ratecourse_row2VBox = widgets.VBox([first_HBox, second_HBox, third_HBox, fourth_HBox, fifth_HBox])
  ratecourse_row3VBox = widgets.VBox([comment_desc, commentcourse_textbox])
  rate_row7VBox = widgets.VBox([input_comment_btn])
  

  display(widgets.VBox([get_course_id_for_sfq_function_output, ratecourse_row1VBox, ratecourse_row2VBox, ratecourse_row3VBox, rate_row7VBox, sfq_result_label]))
##################### SFQ - End#####################
################ Review_I - End ################

############### Dashboard Tab - Begin ################
###########################################################
with tb_m.output_to('Dashboard'):

  # create 1 label 
  display_label = widgets.Label("Choose an item to view")

  # create buttons
  btnmonrev = widgets.Button(description="Revenues")
  btnnewen = widgets.Button(description="New Enrollments")
  btntopcourse = widgets.Button(description="Top Courses")
  btntopinstr = widgets.Button(description="Top Instructors")
  btnnoenrol = widgets.Button(description="Enrollments Num")
  btnavailcourse = widgets.Button(description="Available Courses")

  #susanna defined
  show_mon_rev_output=widgets.Output()
  show_new_enr_output=widgets.Output()
  show_topcourse_output=widgets.Output()
  show_topins_output=widgets.Output()
  show_num_enr_output=widgets.Output()
  show_avail_course_output=widgets.Output()

  manager_dashboard_output_list = [show_mon_rev_output, show_new_enr_output, show_topcourse_output, show_topins_output, show_num_enr_output, show_avail_course_output]
  for item in range(len(manager_dashboard_output_list)):
    manager_dashboard_output_list[item].clear_output()

  # display the objects
  mainMenu_buttons = widgets.GridBox([btnmonrev, btnnewen, btntopcourse, btntopinstr, btnnoenrol, btnavailcourse],
                                      layout=widgets.Layout(grid_template_columns="repeat(3, 1fr)", grid_gap='10px'))

  # Functions to retrieve and display data from the database
  def show_mon_rev(button):
    for item in range(len(manager_dashboard_output_list)):
      manager_dashboard_output_list[item].clear_output()
    
    # code to retrieve and display monthly revenue data from the database
    monrev = []
    monrev_HBox = []
    monrev_HBox.append(widgets.HBox([Text(value = "Monthly Revenue")]))

    dailrev = []
    dailrev_HBox = []
    dailrev_HBox.append(widgets.HBox([Text(value = "Daily Revenue")]))
    def get_mon_rev_function():
      c.execute("""SELECT SUM(c.course_enrollment_fee) 
      FROM ENROLLMENT e
      INNER JOIN COURSE c
      ON e.course_id = c.course_id
      WHERE TO_CHAR(enr_date, 'YYYY-MM') = TO_CHAR(SYSDATE, 'YYYY-MM')""")
      for mon in c:
        monrev.append(Text(value = str(mon[0])))
      for i in range(len(monrev)):
        monrev_HBox.append(widgets.HBox([monrev[i]]))

    def get_dail_rev_function():
      c.execute("""SELECT SUM(c.course_enrollment_fee) 
      FROM ENROLLMENT e
      INNER JOIN COURSE c
      ON e.course_id = c.course_id
      WHERE TO_CHAR(enr_date, 'YYYY-MM-DD') = TO_CHAR(SYSDATE, 'YYYY-MM-DD')""")
      for row in c:
        dailrev.append(Text(value = str(row[0])))
      for i in range(len(dailrev)):
        dailrev_HBox.append(widgets.HBox([dailrev[i]]))

    get_mon_rev_function()
    get_dail_rev_function()

    with show_mon_rev_output:
      if len(monrev_HBox)>1:
        display(widgets.HBox([i for i in monrev_HBox]))
      elif len(monrev_HBox)==1:
        display(widgets.VBox([Text("Monthly revenue will be available soon")]))
      if len(dailrev_HBox)>1:
        display(widgets.HBox([i for i in dailrev_HBox]))
      elif len(dailrev_HBox)==1:
        display(widgets.VBox([Text("Daily revenue will be available soon")]))
  btnmonrev.on_click(show_mon_rev)

  def show_new_enr(button):
    for item in range(len(manager_dashboard_output_list)):
      manager_dashboard_output_list[item].clear_output()
    # code to retrieve and display daily new enrollments data from the database
    newen_top_label = Text(value = "Daily New Enrollments:")
    c.execute("""SELECT e.stu_id, e.course_id, s.stu_name, c.course_code
    FROM ENROLLMENT e
    inner join student s
    on s.stu_id = e.stu_id
    inner join course c
    on c.course_id = e.course_id
    WHERE TO_CHAR(enr_date, 'YYYY-MM-DD') = TO_CHAR(SYSDATE, 'YYYY-MM-DD')""")
    newen_sid= []
    newen_cid= []
    newen_stuname= []
    newen_coursecode= []
    newen_HBox= []
    newen_HBox.append(widgets.HBox([Text(value = "Student ID"), Text(value = "Course ID"), Text(value = "Student Name"), Text(value = "Course Code")]))

    for row in c:
      newen_sid.append(Text(value = str(row[0])))
      newen_cid.append(Text(value = str(row[1])))
      newen_stuname.append(Text(value = str(row[2])))
      newen_coursecode.append(Text(value = str(row[3])))
    for i in range(len(newen_sid)):
      newen_HBox.append(widgets.HBox([newen_sid[i], newen_cid[i], newen_stuname[i], newen_coursecode[i]]))

    with show_new_enr_output:
      display(widgets.VBox([newen_top_label]))
      display(widgets.VBox([i for i in newen_HBox]))
  btnnewen.on_click(show_new_enr)

  def show_topcourse(button):
    for item in range(len(manager_dashboard_output_list)):
      manager_dashboard_output_list[item].clear_output()
    show_topcourse_top_label = Text(value= "Top Courses with most enrollment: ")
    # code to retrieve and display top courses data from the database
    topcour_id= []
    topcour_enr=[]
    topcour_HBox = []
    topcour_HBox.append(widgets.HBox([Text(value = "Top course ID"), Text(value = "Number of enrollments")]))

    c.execute("""
        SELECT COURSE.course_id, COUNT(*)
            FROM COURSE
            INNER JOIN ENROLLMENT ON COURSE.course_id = ENROLLMENT.course_id
            GROUP BY COURSE.course_id
            ORDER BY COUNT(*) DESC
    """)

    for row in c:
      topcour_id.append(Text(value=str(row[0])))
      topcour_enr.append(Text(value=str(row[1])))
    if len(topcour_id)>0:
      for i in range(len(topcour_id)):
        topcour_HBox.append(widgets.HBox([topcour_id[i], topcour_enr[i]]))
    elif len(topcour_id)==0:
      show_topcourse_top_label = Text("More information will be available soon!")

    with show_topcourse_output:
      display(widgets.VBox([show_topcourse_top_label]))
      if len(topcour_id)>0:
        display(widgets.VBox([i for i in topcour_HBox]))
  btntopcourse.on_click(show_topcourse)


  def show_topins(button):
    for item in range(len(manager_dashboard_output_list)):
      manager_dashboard_output_list[item].clear_output()
    show_topins_top_label = Text(value="Instructors with highest ratings: ")
    # code to retrieve and display top instructors data from the database
    topinstr_name= []
    topinstr_id= []
    topinstr_fb_rating= []
    topinstr_HBox= []
    topinstr_HBox.append(widgets.HBox([Text(value = "Instructor ID"), Text(value = "Instructor Name"), Text(value = "Feedback rating")]))
    c.execute(""" SELECT INSTRUCTOR.ins_id, INSTRUCTOR.ins_name, AVG(FEEDBACK.fb_rating)
    FROM INSTRUCTOR 
    INNER JOIN FEEDBACK
    ON INSTRUCTOR.ins_id = FEEDBACK.ins_id
    GROUP BY INSTRUCTOR.ins_id, INSTRUCTOR.ins_name
    ORDER BY AVG(FEEDBACK.fb_rating) DESC
    """)
    
    for row in c:
      topinstr_name.append(Text(value=str(row[0])))
      topinstr_id.append(Text(value=str(row[1])))
      topinstr_fb_rating.append(Text(value=str(row[2])))
    if len(topinstr_id)>0:
      for i in range(len(topinstr_id)):
        topinstr_HBox.append(widgets.HBox([topinstr_name[i], topinstr_id[i], topinstr_fb_rating[i]]))
    elif len(topinstr_id)==0:
      show_topins_top_label = Text("Top instructors ranking is not available right now. More information will be released soon!")

    with show_topins_output:
      display(widgets.VBox([show_topins_top_label]))
      if len(topinstr_id)>0:
        display(widgets.VBox([i for i in topinstr_HBox]))
  btntopinstr.on_click(show_topins)

  def show_num_enr(button):
    for item in range(len(manager_dashboard_output_list)):
      manager_dashboard_output_list[item].clear_output()
    # code to retrieve and display number of enrollments data from the database
    show_num_enr_top_label = Text(value="Course Status: ")
    course_enr_status = []
    noenr= []
    show_num_enr_HBox = []
    show_num_enr_HBox.append(widgets.HBox([Text(value = "Course Status"), Text(value = "Number of enrollments")]))
    c.execute("SELECT course_status, COUNT(*) FROM COURSE GROUP BY course_status")
    
    for row in c:
      course_enr_status.append(Text(value=str(row[0])))
      noenr.append(Text(value=str(row[1])))
    
    if len(course_enr_status) >0:
      for i in range(len(course_enr_status)):
        show_num_enr_HBox.append(widgets.HBox([course_enr_status[i], noenr[i]]))
    elif len(course_enr_status) ==0:
      show_num_enr_top_label = Text("More information will be released soon!")
    with show_num_enr_output:
      display(widgets.VBox([show_num_enr_top_label]))
      if len(course_enr_status) >0:
        display(widgets.VBox([i for i in show_num_enr_HBox]))
  btnnoenrol.on_click(show_num_enr)

  def show_avail_course(button):
    for item in range(len(manager_dashboard_output_list)):
      manager_dashboard_output_list[item].clear_output()
    # code to retrieve and display available courses data from the database
    show_avail_course_top_label = Text("Current Available Courses:")
    avail_course_id = []
    avail_course_name = []
    availenr= []
    avail_course_HBox = []
    avail_course_HBox.append(widgets.HBox([Text(value = "Course ID"), Text(value = "Course Name")]))
    c.execute("SELECT course_id, course_name, course_avail FROM COURSE WHERE course_quota > course_enrol AND course_status = 'open'")
    for row in c:
      avail_course_id.append(Text(value=str(row[0])))
      avail_course_name.append(Text(value=str(row[1])))
      availenr.append(Text(value=str(row[2])))
    
    if len(avail_course_id) >0:
      for i in range(len(avail_course_id)):
        avail_course_HBox.append(widgets.HBox([avail_course_id[i], avail_course_name[i]]))
    elif  len(avail_course_id) ==0:
      show_avail_course_top_label = Text("No current available course.")
    with show_avail_course_output:
      display(widgets.VBox([show_avail_course_top_label]))
      if len(avail_course_id) >0:
        display(widgets.VBox([i for i in avail_course_HBox]))

  btnavailcourse.on_click(show_avail_course)

  display(widgets.VBox([display_label, mainMenu_buttons, spacer, show_mon_rev_output, show_new_enr_output, show_topcourse_output, show_topins_output, show_num_enr_output, show_avail_course_output]))

###########dashboard end##########

###########################################################
##################### Manager Home Page (Ming)#####################
###########################################################

###########################################################
##################### Account Maintenance (Ming)####################
###########################################################

###########################################################
##################### Update/Delete Student (Ming)####################
###########################################################
with tb_m_acmaintain.output_to('Update/Delete Student'):
  MAN_MAIN_STU_output = widgets.Output()
  #create 8 textbox
  MAN_MAIN_STU_id_textbox = Text(description = 'Student ID*:', disabled=True)
  MAN_MAIN_STU_name_textbox = Text(description='Name*:', disabled=False)
  MAN_MAIN_STU_username_textbox = Text(description='Username*:', disabled=False)
  MAN_MAIN_STU_PWD_password = Password(description='Password*:', disabled=False)
  MAN_MAIN_STU_email_textbox = Text(description='Email*:', disabled=False)
  MAN_MAIN_STU_address_textbox = Text(description='Address*:', disabled=False)
  MAN_MAIN_STU_job_textbox = Text(description='Job:', disabled=False)
  MAN_MAIN_STU_qualification_textbox = Text(description='Qualification:', disabled=False)
  
  #create 4 button
  MAN_MAIN_STU_retrieve_button = widgets.Button(description='Retrieve')
  MAN_MAIN_STU_update_button = widgets.Button(description='Update')
  MAN_MAIN_STU_delete_button = widgets.Button(description='Delete')
  MAN_MAIN_STU_clear_button = widgets.Button(description='Clear')

  #create two lists
  MAN_STU_list = []
  MAN_STU_id_list = []
  MAN_MAIN_STU_dropdown = widgets.Dropdown()

  #get student list
  def MAN_STU_LIST_GET():
    global MAN_STU_list, MAN_STU_id_list
    MAN_STU_list = []
    MAN_STU_id_list = []
    c.execute("""SELECT stu_id,
              stu_name
          FROM STUDENT
          ORDER BY stu_name""")
    for row in c:
      MAN_STU_list.append(row[1] + '(' + str(row[0]) + ')')
      MAN_STU_id_list.append(row[0])
    #Update drop down menu
    MAN_MAIN_STU_dropdown.options = MAN_STU_list

  MAN_STU_LIST_GET()

  def on_MAN_MAIN_STU_retrieve_button_clicked(b):
    MAN_MAIN_STU_output.clear_output()
    if len(MAN_STU_id_list) == 0:
      with MAN_MAIN_STU_output:
        display(widgets.Label('No student is found!'))
    else:
      index_no = MAN_STU_list.index(MAN_MAIN_STU_dropdown.value)
      c.execute("""SELECT *
            FROM STUDENT
            WHERE stu_id = '""" + str(MAN_STU_id_list[index_no]) + "'")
      # retrieve data from database and assign to the textbox
      for row in c:
        MAN_MAIN_STU_id_textbox.value = str(row[0])
        MAN_MAIN_STU_name_textbox.value = row[1]    
        MAN_MAIN_STU_username_textbox.value = row[2]
        MAN_MAIN_STU_PWD_password.value = row[3]
        MAN_MAIN_STU_email_textbox.value = row[4]
        MAN_MAIN_STU_address_textbox.value = row[5]
        if row[6] != None:
          MAN_MAIN_STU_job_textbox.value = row[6]
        else:
          MAN_MAIN_STU_job_textbox.value = ''
        if row[7] != None:
          MAN_MAIN_STU_qualification_textbox.value = row[7]
        else:
          MAN_MAIN_STU_qualification_textbox.value = ''
  MAN_MAIN_STU_retrieve_button.on_click(on_MAN_MAIN_STU_retrieve_button_clicked)

  def on_MAN_MAIN_STU_update_button_clicked(b):
    MAN_MAIN_STU_output.clear_output()
    if MAN_MAIN_STU_id_textbox.value == '':
      with MAN_MAIN_STU_output:
        display(widgets.Label('Please select and retrieve a Student before Updating!'))
    elif MAN_MAIN_STU_name_textbox.value==''or MAN_MAIN_STU_username_textbox.value=='' or MAN_MAIN_STU_PWD_password.value=='' or MAN_MAIN_STU_email_textbox.value=='' or MAN_MAIN_STU_address_textbox.value=='':
      with MAN_MAIN_STU_output:
        display(widgets.Label('Please input fields with "*" indicated!'))
    elif not(re.match(r"[^@]+@[^@]+\.[^@]+", MAN_MAIN_STU_email_textbox.value)):
      with MAN_MAIN_STU_output:
        display(widgets.Label('Invalid Email!'))
    else:
      c.execute(f"SELECT stu_id FROM STUDENT WHERE stu_username ='{MAN_MAIN_STU_username_textbox.value}'")
      stuid = -1
      for row in c:
        stuid = row[0]
      if stuid == -1 or str(stuid) == MAN_MAIN_STU_id_textbox.value:
        c.execute(
        """
          UPDATE STUDENT 
          SET stu_name = '{}',
          stu_username = '{}',
          stu_pwd = '{}',
          stu_email = '{}',
          stu_address = '{}',
          stu_job = '{}',
          stu_qualification = '{}'
          WHERE stu_id = '{}'
        """.format(MAN_MAIN_STU_name_textbox.value, MAN_MAIN_STU_username_textbox.value,
              MAN_MAIN_STU_PWD_password.value, MAN_MAIN_STU_email_textbox.value,
              MAN_MAIN_STU_address_textbox.value, MAN_MAIN_STU_job_textbox.value,
              MAN_MAIN_STU_qualification_textbox.value, MAN_MAIN_STU_id_textbox.value)
        )
        c.execute("commit")
        with MAN_MAIN_STU_output:
          display(widgets.Label('Update Success!'))
        MAN_STU_LIST_GET() #Update drop down menu
      else:
        with MAN_MAIN_STU_output:
          display(widgets.Label('Another user already using the same username. Please choose anotehr one!'))
  MAN_MAIN_STU_update_button.on_click(on_MAN_MAIN_STU_update_button_clicked)

  def on_MAN_MAIN_STU_delete_button_clicked(b):
    MAN_MAIN_STU_output.clear_output()
    if MAN_MAIN_STU_id_textbox.value != '':
      c.execute(
      """
            DELETE FROM STUDENT 
            WHERE stu_id = '{}'
      """.format(MAN_MAIN_STU_id_textbox.value)
      )
      c.execute("commit")
      MAN_MAIN_STU_clear_button.click()
      MAN_STU_LIST_GET() #Update drop down menu
      with MAN_MAIN_STU_output:
        display(widgets.Label('Delete Success!'))
    else:
      with MAN_MAIN_STU_output:
        display(widgets.Label('Please select and retrieve a Student before Deleting!'))
  MAN_MAIN_STU_delete_button.on_click(on_MAN_MAIN_STU_delete_button_clicked)

  def on_MAN_MAIN_STU_clear_button_clicked(b):
    MAN_MAIN_STU_output.clear_output()
    MAN_MAIN_STU_id_textbox.value = ""
    MAN_MAIN_STU_name_textbox.value = ""
    MAN_MAIN_STU_username_textbox.value = ""
    MAN_MAIN_STU_PWD_password.value = ""
    MAN_MAIN_STU_email_textbox.value = ""
    MAN_MAIN_STU_address_textbox.value = ""
    MAN_MAIN_STU_job_textbox.value = ""
    MAN_MAIN_STU_qualification_textbox.value = ""
  MAN_MAIN_STU_clear_button.on_click(on_MAN_MAIN_STU_clear_button_clicked)

  stumain1HBox = widgets.HBox([MAN_MAIN_STU_dropdown, MAN_MAIN_STU_retrieve_button])
  stumain1aHBox = widgets.HBox([spacer])
  stumain2HBox = widgets.HBox([MAN_MAIN_STU_id_textbox])
  stumain3HBox = widgets.HBox([MAN_MAIN_STU_name_textbox, MAN_MAIN_STU_username_textbox])
  stumain4HBox = widgets.HBox([MAN_MAIN_STU_PWD_password, MAN_MAIN_STU_email_textbox])
  stumain5HBox = widgets.HBox([MAN_MAIN_STU_address_textbox, MAN_MAIN_STU_job_textbox])
  stumain6HBox = widgets.HBox([MAN_MAIN_STU_qualification_textbox])
  stumain7HBox = widgets.HBox([MAN_MAIN_STU_update_button, MAN_MAIN_STU_delete_button])
  stumain8HBox = widgets.HBox([MAN_MAIN_STU_clear_button])
  stumain9HBox = widgets.HBox([MAN_MAIN_STU_output])

  display(widgets.VBox([stumain1HBox, stumain1aHBox, stumain2HBox, stumain3HBox, stumain4HBox, stumain5HBox, stumain6HBox, stumain7HBox, stumain8HBox, stumain9HBox]))
##################### Update/Delete Student - End ####################


###########################################################
##################### Insert Student (Ming)####################
###########################################################
with tb_m_acmaintain.output_to('Insert Student'):
  MAN_INS_STU_output = widgets.Output()
  #create 7 textbox
  MAN_INS_STU_name_textbox = Text(description='Name*:', disabled=False)
  MAN_INS_STU_username_textbox = Text(description='Username*:', disabled=False)
  MAN_INS_STU_PWD_password = Password(description='Password*:', disabled=False)
  MAN_INS_STU_email_textbox = Text(description='Email*:', disabled=False)
  MAN_INS_STU_address_textbox = Text(description='Address*:', disabled=False)
  MAN_INS_STU_job_textbox = Text(description='Job:', disabled=False)
  MAN_INS_STU_qualification_textbox = Text(description='Qualification:', disabled=False)

  #create 2 button
  MAN_INS_STU_insert_button = widgets.Button(description='Insert')
  MAN_INS_STU_clear_button = widgets.Button(description='Clear')

  def on_MAN_INS_STU_insert_button_clicked(b):
    MAN_INS_STU_output.clear_output()
    if MAN_INS_STU_name_textbox.value==''or MAN_INS_STU_username_textbox.value=='' or MAN_INS_STU_PWD_password.value=='' or MAN_INS_STU_email_textbox.value=='' or MAN_INS_STU_address_textbox.value=='':
      with MAN_INS_STU_output:
        display(widgets.Label('Please input fields with "*" indicated!'))
    elif not(re.match(r"[^@]+@[^@]+\.[^@]+", MAN_INS_STU_email_textbox.value)):
      with MAN_INS_STU_output:
        display(widgets.Label('Invalid Email!'))
    else:
      c.execute(f"SELECT stu_username FROM STUDENT WHERE stu_username ='{MAN_INS_STU_username_textbox.value}'")
      stu_username = ''
      for row in c:
        stu_username = row[0]
      if stu_username == '':
        c.execute("""SELECT max(STU_ID) FROM STUDENT""")
        for row in c:
          if row[0] != None:
            stu_id_max = row[0]
          else:
            stu_id_max = 0
        c.execute(
        """
              INSERT INTO STUDENT VALUES ({},
                              '{}',
                              '{}',
                              '{}',
                              '{}',
                              '{}',
                              '{}',
                              '{}')
        """.format(stu_id_max+1,
              MAN_INS_STU_name_textbox.value,
              MAN_INS_STU_username_textbox.value,
              MAN_INS_STU_PWD_password.value,
              MAN_INS_STU_email_textbox.value,
              MAN_INS_STU_address_textbox.value,
              MAN_INS_STU_job_textbox.value,
              MAN_INS_STU_qualification_textbox.value)
        )
        c.execute("commit")
        MAN_INS_STU_clear_button.click()
        MAN_STU_LIST_GET() #Update drop down menu
        with MAN_INS_STU_output:
          display(widgets.Label('Insert Success!'))
      else:
        with MAN_INS_STU_output:
          display(widgets.Label('Another user already using the same username. Please choose anotehr one!'))
  MAN_INS_STU_insert_button.on_click(on_MAN_INS_STU_insert_button_clicked)

  def on_MAN_INS_STU_clear_button_clicked(b):
    MAN_INS_STU_output.clear_output()
    MAN_INS_STU_name_textbox.value = ""
    MAN_INS_STU_username_textbox.value = ""
    MAN_INS_STU_PWD_password.value = ""
    MAN_INS_STU_email_textbox.value = ""
    MAN_INS_STU_address_textbox.value = ""
    MAN_INS_STU_job_textbox.value = ""
    MAN_INS_STU_qualification_textbox.value = ""
  MAN_INS_STU_clear_button.on_click(on_MAN_INS_STU_clear_button_clicked)

  stuinsert1HBox = widgets.HBox([MAN_INS_STU_name_textbox])
  stuinsert2HBox = widgets.HBox([MAN_INS_STU_username_textbox])
  stuinsert3HBox = widgets.HBox([MAN_INS_STU_PWD_password])
  stuinsert4HBox = widgets.HBox([MAN_INS_STU_email_textbox])
  stuinsert5HBox = widgets.HBox([MAN_INS_STU_address_textbox])
  stuinsert6HBox = widgets.HBox([MAN_INS_STU_job_textbox])
  stuinsert7HBox = widgets.HBox([MAN_INS_STU_qualification_textbox])
  stuinsert8HBox = widgets.HBox([MAN_INS_STU_insert_button, MAN_INS_STU_clear_button])
  stuinsert9HBox = widgets.HBox([MAN_INS_STU_output])

  display(widgets.VBox([stuinsert1HBox, stuinsert2HBox, stuinsert3HBox, stuinsert4HBox, stuinsert5HBox, stuinsert6HBox, stuinsert7HBox, stuinsert8HBox, stuinsert9HBox]))

##################### Insert Student - End ####################

###########################################################
##################### Update/Delete Instructor (Ming)####################
###########################################################


with tb_m_acmaintain.output_to('Update/Delete Instructor'):
  #create 7 textbox
  MAN_MAIN_INST_output = widgets.Output()

  MAN_MAIN_INST_id_textbox = Text(disabled=True)
  MAN_MAIN_INST_name_textbox = Text(disabled=False)
  MAN_MAIN_INST_username_textbox = Text(disabled=False)
  MAN_MAIN_INST_PWD_password = Password(disabled=False)
  MAN_MAIN_INST_email_textbox = Text(disabled=False)
  MAN_MAIN_INST_edu_bg_textbox = Text(disabled=False)
  MAN_MAIN_INST_bio_textbox = Text(disabled=False)

  #create 7 label
  MAN_MAIN_INST_label_layout = Layout(width='160px')

  MAN_INST_id_label = Label('Instructor*:', layout=MAN_MAIN_INST_label_layout)
  MAN_INST_name_label = Label('Name*:', layout=MAN_MAIN_INST_label_layout)
  MAN_INST_username_label = Label('Username*:', layout=MAN_MAIN_INST_label_layout)
  MAN_INST_PWD_label = Label('Password*:', layout=MAN_MAIN_INST_label_layout)
  MAN_INST_email_label = Label('Email*:', layout=MAN_MAIN_INST_label_layout)
  MAN_INST_edu_bg_label = Label('Education Background:', layout=MAN_MAIN_INST_label_layout)
  MAN_INST_bio_label = Label('Biography:', layout=MAN_MAIN_INST_label_layout)

  #create 4 button
  MAN_MAIN_INST_retrieve_button = widgets.Button(description='Retrieve')
  MAN_MAIN_INST_update_button = widgets.Button(description='Update')
  MAN_MAIN_INST_delete_button = widgets.Button(description='Delete')
  MAN_MAIN_INST_clear_button = widgets.Button(description='Clear')

  #create instructor list
  MAN_INST_list = []
  MAN_INST_id_list = []

  #create dropdown
  MAN_MAIN_INST_dropdown = widgets.Dropdown()

  def MAN_INST_LIST_GET():
    global MAN_INST_list, MAN_INST_id_list
    MAN_INST_list = []
    MAN_INST_id_list = []
    c.execute("""SELECT ins_id,
              ins_name
          FROM INSTRUCTOR
          ORDER BY ins_name""")
    for row in c:
      MAN_INST_list.append(row[1] + '(' + str(row[0]) + ')')
      MAN_INST_id_list.append(row[0])
    #create drop down menu
    MAN_MAIN_INST_dropdown.options = MAN_INST_list
  MAN_INST_LIST_GET()

  def on_MAN_MAIN_INST_retrieve_button_clicked(b):
    MAN_MAIN_INST_output.clear_output()
    if len(MAN_INST_id_list)==0:
      with MAN_MAIN_INST_output:
        display(widgets.Label('No instructor record found in Database!'))
    else:
      index_no = MAN_INST_list.index(MAN_MAIN_INST_dropdown.value)
      c.execute("""SELECT *
            FROM INSTRUCTOR
            WHERE Ins_id = '""" +  str(MAN_INST_id_list[index_no]) + "'")
      for row in c: # retrieve data from database and assign to the textbox
        MAN_MAIN_INST_id_textbox.value = str(row[0])
        MAN_MAIN_INST_name_textbox.value = row[1]
        MAN_MAIN_INST_username_textbox.value = row[2]
        MAN_MAIN_INST_PWD_password.value = row[3]
        MAN_MAIN_INST_email_textbox.value = row[4]
        if row[5] != None:
          MAN_MAIN_INST_edu_bg_textbox.value = row[5]
        else:
          MAN_MAIN_INST_edu_bg_textbox.value = ''
        if row[6] != None:
          MAN_MAIN_INST_bio_textbox.value = row[6]
        else:
          MAN_MAIN_INST_bio_textbox.value = ''
  MAN_MAIN_INST_retrieve_button.on_click(on_MAN_MAIN_INST_retrieve_button_clicked)

  def on_MAN_MAIN_INST_update_button_clicked(b):
    MAN_MAIN_INST_output.clear_output()
    if MAN_MAIN_INST_id_textbox.value =='':
      with MAN_MAIN_INST_output:
        display(widgets.Label('Please select and retrieve an Instructor before Updating!'))
    elif MAN_MAIN_INST_name_textbox.value=='' or MAN_MAIN_INST_username_textbox.value=='' or MAN_MAIN_INST_PWD_password.value=='' or MAN_MAIN_INST_email_textbox.value=='':
      with MAN_MAIN_INST_output:
        display(widgets.Label('Please input fields with "*" indicated!'))
    elif not(re.match(r"[^@]+@[^@]+\.[^@]+", MAN_MAIN_INST_email_textbox.value)):
      with MAN_MAIN_INST_output:
        display(widgets.Label('Invalid Email!'))
    else:
      c.execute(
      """
        UPDATE INSTRUCTOR 
        SET ins_name = '{}',
        ins_username = '{}',
        ins_pwd = '{}',
        ins_email = '{}',
        ins_education_bg = '{}',
        ins_biography = '{}'
        WHERE ins_id = '{}'
      """.format(MAN_MAIN_INST_name_textbox.value,
            MAN_MAIN_INST_username_textbox.value,
            MAN_MAIN_INST_PWD_password.value,
            MAN_MAIN_INST_email_textbox.value,
            MAN_MAIN_INST_edu_bg_textbox.value,
            MAN_MAIN_INST_bio_textbox.value,
            MAN_MAIN_INST_id_textbox.value)
      )
      c.execute("commit")
      MAN_INST_LIST_GET()
      with MAN_MAIN_INST_output:
        display(widgets.Label('Update Success!'))
  MAN_MAIN_INST_update_button.on_click(on_MAN_MAIN_INST_update_button_clicked)

  def on_MAN_MAIN_INST_delete_button_clicked(b):
    MAN_MAIN_INST_output.clear_output()
    if MAN_MAIN_INST_id_textbox.value !='':
      c.execute(
      """
            DELETE FROM INSTRUCTOR 
            WHERE ins_id = '{}'
      """.format(MAN_MAIN_INST_id_textbox.value)
      )
      c.execute("commit")
      MAN_MAIN_INST_clear_button.click()
      MAN_INST_LIST_GET()
      with MAN_MAIN_INST_output:
        display(widgets.Label('Delete Success!'))
    else:
      with MAN_MAIN_INST_output:
        display(widgets.Label('Please select and retrieve an Instructor before Deleteing!'))
  MAN_MAIN_INST_delete_button.on_click(on_MAN_MAIN_INST_delete_button_clicked)

  def on_MAN_MAIN_INST_clear_button_clicked(b):
    MAN_MAIN_INST_output.clear_output()
    MAN_MAIN_INST_id_textbox.value = ""
    MAN_MAIN_INST_name_textbox.value = ""
    MAN_MAIN_INST_username_textbox.value = ""
    MAN_MAIN_INST_PWD_password.value = ""
    MAN_MAIN_INST_email_textbox.value = ""
    MAN_MAIN_INST_edu_bg_textbox.value = ""
    MAN_MAIN_INST_bio_textbox.value = ""
  MAN_MAIN_INST_clear_button.on_click(on_MAN_MAIN_INST_clear_button_clicked)

  insmain1HBox = widgets.HBox([MAN_MAIN_INST_dropdown, MAN_MAIN_INST_retrieve_button])
  insmain1aHBox = widgets.HBox([spacer])
  insmain2HBox = widgets.HBox([MAN_INST_id_label, MAN_MAIN_INST_id_textbox])
  insmain3HBox = widgets.HBox([MAN_INST_name_label, MAN_MAIN_INST_name_textbox])
  insmain4HBox = widgets.HBox([MAN_INST_username_label, MAN_MAIN_INST_username_textbox])
  insmain5HBox = widgets.HBox([MAN_INST_PWD_label, MAN_MAIN_INST_PWD_password])
  insmain6HBox = widgets.HBox([MAN_INST_email_label, MAN_MAIN_INST_email_textbox])
  insmain7HBox = widgets.HBox([MAN_INST_edu_bg_label, MAN_MAIN_INST_edu_bg_textbox])
  insmain8HBox = widgets.HBox([MAN_INST_bio_label, MAN_MAIN_INST_bio_textbox])
  insmain9HBox = widgets.HBox([MAN_MAIN_INST_update_button, MAN_MAIN_INST_delete_button])
  insmain10HBox = widgets.HBox([MAN_MAIN_INST_clear_button])
  insmain11HBox = widgets.HBox([MAN_MAIN_INST_output])
  display(widgets.VBox([insmain1HBox,
            insmain1aHBox,
            insmain2HBox,
            insmain3HBox,
            insmain4HBox,
            insmain5HBox,
            insmain6HBox,
            insmain7HBox,
            insmain8HBox,
            insmain9HBox,
            insmain10HBox,
            insmain11HBox]))
  
##################### Update/Delete Instructor - End ####################

###########################################################
##################### Insert Instructor (Ming)####################
###########################################################

with tb_m_acmaintain.output_to('Insert Instructor'):
  MAN_INS_INST_output = widgets.Output()
  MAN_INS_INST_label_layout = Layout(width='160px')
  
  #create 6 textbox
  MAN_INS_INST_name_textbox = Text(disabled=False)
  MAN_INS_INST_username_textbox = Text(disable=False)
  MAN_INS_INST_PWD_password = Password(disabled=False)
  MAN_INS_INST_email_textbox = Text(disabled=False)
  MAN_INS_INST_edu_bg_textbox = Text(disabled=False)
  MAN_INS_INST_bio_textbox = Text(disabled=False)

  #create buttons
  MAN_INS_INST_insert_button = widgets.Button(description='Insert')
  MAN_INS_INST_clear_button = widgets.Button(description='Clear')

  def on_MAN_INS_INST_insert_button_clicked(b):
    MAN_INS_INST_output.clear_output()
    if MAN_INS_INST_name_textbox.value=='' or MAN_INS_INST_username_textbox.value=='' or MAN_INS_INST_PWD_password.value=='' or MAN_INS_INST_email_textbox.value=='':
      with MAN_INS_INST_output:
        display(widgets.Label('Please input fields with "*" indicated!'))
    elif not(re.match(r"[^@]+@[^@]+\.[^@]+", MAN_INS_INST_email_textbox.value)):
      with MAN_INS_INST_output:
        display(widgets.Label('Invalid Email!'))
    else:
      c.execute("""SELECT max(INS_ID) FROM INSTRUCTOR""")
      for row in c:
        if row[0] != None:
          ins_id_max = row[0]
        else:
          ins_id_max = 0
      c.execute(
      """
            INSERT INTO INSTRUCTOR VALUES ({},
                              '{}',
                              '{}',
                              '{}',
                              '{}',
                              '{}',
                              '{}')
      """.format(ins_id_max+1,
          MAN_INS_INST_name_textbox.value,
          MAN_INS_INST_username_textbox.value,
          MAN_INS_INST_PWD_password.value,
          MAN_INS_INST_email_textbox.value,
          MAN_INS_INST_edu_bg_textbox.value,
          MAN_INS_INST_bio_textbox.value)
      )
      c.execute("commit")
      MAN_INS_INST_clear_button.click()
      MAN_INST_LIST_GET()
      with MAN_INS_INST_output:
        display(widgets.Label('Insert Success!'))
  MAN_INS_INST_insert_button.on_click(on_MAN_INS_INST_insert_button_clicked)

  def on_MAN_INS_INST_clear_button_clicked(b):
    MAN_INS_INST_output.clear_output()
    MAN_INS_INST_name_textbox.value = ""
    MAN_INS_INST_username_textbox.value = ""
    MAN_INS_INST_PWD_password.value = ""
    MAN_INS_INST_email_textbox.value = ""
    MAN_INS_INST_edu_bg_textbox.value = ""
    MAN_INS_INST_bio_textbox.value = ""
  MAN_INS_INST_clear_button.on_click(on_MAN_INS_INST_clear_button_clicked)

  instinsert1HBox = widgets.HBox([MAN_INST_name_label, MAN_INS_INST_name_textbox])
  instinsert2HBox = widgets.HBox([MAN_INST_username_label, MAN_INS_INST_username_textbox])
  instinsert3HBox = widgets.HBox([MAN_INST_PWD_label, MAN_INS_INST_PWD_password])
  instinsert4HBox = widgets.HBox([MAN_INST_email_label, MAN_INS_INST_email_textbox])
  instinsert5HBox = widgets.HBox([MAN_INST_edu_bg_label, MAN_INS_INST_edu_bg_textbox])
  instinsert6HBox = widgets.HBox([MAN_INST_bio_label, MAN_INS_INST_bio_textbox])
  instinsert7HBox = widgets.HBox([MAN_INS_INST_insert_button, MAN_INS_INST_clear_button])
  instinsert8HBox = widgets.HBox([MAN_INS_INST_output])

  display(widgets.VBox([instinsert1HBox,
              instinsert2HBox,
              instinsert3HBox,
              instinsert4HBox,
              instinsert5HBox,
              instinsert6HBox,
              instinsert7HBox,
              instinsert8HBox]))
################ Insert Instructor - End ################

##################### Account Maintenance - End ####################

###########################################################
##################### Course (Ming)####################
###########################################################

###########################################################
##################### View Course (Ming)####################
###########################################################
with tb_m_course.output_to('View Course'):
  MAN_VIEW_COURSE_output = widgets.Output()
  #create textbox
  MAN_VIEW_COURSE_id_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_INS_id_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_code_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_name_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_credit_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_section_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_day_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_starttime_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_endtime_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_room_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_instructor_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_quota_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_enrol_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_avail_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_wait_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_desc_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_enrollment_fee_textbox = Text(disabled=True)
  MAN_VIEW_COURSE_startdate_datepicker = DatePicker(disabled=True)
  MAN_VIEW_COURSE_enddate_datepicker = DatePicker(disabled=True)
  MAN_VIEW_COURSE_status_dropdown = Dropdown(disabled=True, options=['open', 'closed', 'cancelled'])

  #create label
  MAN_COURSE_label_layout = Layout(width='100px')

  MAN_COURSE_id_label = Label('Course ID*:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_INS_id_label = Label('Instructor ID*:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_code_label = Label('Code*:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_name_label = Label('Name*:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_credit_label = Label('Credit*:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_section_label = Label('Section:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_day_label = Label('Day:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_starttime_label = Label('Start Time:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_endtime_label = Label('End Time:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_room_label = Label('Room:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_instructor_label = Label('Instructor:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_quota_label = Label('Quota:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_enrol_label = Label('Enrolled:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_avail_label = Label('Available:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_wait_label = Label('Waiting:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_desc_label = Label('Desc:', layout=MAN_COURSE_label_layout) #pending for change
  MAN_COURSE_enrollment_fee_label = Label('Enrollment Fee:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_startdate_label = Label('Start Date:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_enddate_label = Label('End Date:', layout=MAN_COURSE_label_layout)
  MAN_COURSE_status_label = Label('Status*:', layout=MAN_COURSE_label_layout)

  #create 2 button
  MAN_VIEW_COURSE_retrieve_button = widgets.Button(description='Retrieve')
  MAN_VIEW_COURSE_clear_button = widgets.Button(description='Clear')

  #create course list
  MAN_COURSE_list = []
  MAN_COURSE_id_list = []
  
  #create dropdown
  MAN_VIEW_COURSE_dropdown = widgets.Dropdown()
  MAN_MAIN_COURSE_dropdown = widgets.Dropdown()
  MAN_ROSTER_COURSE_dropdown = widgets.Dropdown()

  #get student list
  def MAN_COURSE_LIST_GET():
    global MAN_COURSE_list, MAN_COURSE_id_list
    MAN_COURSE_list = []
    MAN_COURSE_id_list = []
    c.execute("""SELECT course_id,
              course_code,
              course_name,
              course_section,
              course_instructor
          FROM COURSE
          ORDER BY course_code, course_section""")
    for row in c:
      MAN_COURSE_list.append(f"{str(row[1])}({row[3]})({row[4]}) - ({str(row[0])})")
      MAN_COURSE_id_list.append(str(row[0]))
    #Update dropdown menu
    MAN_VIEW_COURSE_dropdown.options = MAN_COURSE_list
    MAN_MAIN_COURSE_dropdown.options = MAN_COURSE_list
    MAN_ROSTER_COURSE_dropdown.options = MAN_COURSE_list
  MAN_COURSE_LIST_GET()

  def on_MAN_VIEW_COURSE_retrieve_button_clicked(b):
    MAN_VIEW_COURSE_output.clear_output()
    if len(MAN_COURSE_id_list)==0:
      with MAN_VIEW_COURSE_output:
        display(widgets.Label('No course record found in Database!'))
    else:
      index_no = MAN_COURSE_list.index(MAN_VIEW_COURSE_dropdown.value)
      c.execute("""SELECT *
            FROM COURSE
            WHERE course_id = '""" + str(MAN_COURSE_id_list[index_no]) + "'")
      #retrieve data from database and assign to the textbox
      for row in c:
        MAN_VIEW_COURSE_id_textbox.value = str(row[0])
        MAN_VIEW_COURSE_INS_id_textbox.value = str(row[1])    
        MAN_VIEW_COURSE_code_textbox.value = row[2]
        MAN_VIEW_COURSE_name_textbox.value = row[3]
        MAN_VIEW_COURSE_credit_textbox.value = str(row[4])
        if row[5] != None:
          MAN_VIEW_COURSE_section_textbox.value = row[5]
        else:
          MAN_VIEW_COURSE_section_textbox.value = ''
        if row[6] != None:
          MAN_VIEW_COURSE_day_textbox.value = row[6]
        else:
          MAN_VIEW_COURSE_day_textbox.value = ''
        if row[7] != None:
          MAN_VIEW_COURSE_starttime_textbox.value = row[7]
        else:
          MAN_VIEW_COURSE_starttime_textbox.value = ''
        if row[8] != None:
          MAN_VIEW_COURSE_endtime_textbox.value = row[8]
        else:
          MAN_VIEW_COURSE_endtime_textbox.value = ''
        if row[9] != None:
          MAN_VIEW_COURSE_room_textbox.value = row[9]
        else:
          MAN_VIEW_COURSE_room_textbox.value = ''
        if row[10] != None:
          MAN_VIEW_COURSE_instructor_textbox.value = row[10]
        else:
          MAN_VIEW_COURSE_instructor_textbox.value = ''
        if row[11] != None:
          MAN_VIEW_COURSE_quota_textbox.value = str(row[11])
        else:
          MAN_VIEW_COURSE_quota_textbox.value = ''
        if row[12] != None:
          MAN_VIEW_COURSE_enrol_textbox.value = str(row[12])
        else:
          MAN_VIEW_COURSE_enrol_textbox.value = ''
        if row[13] != None:
          MAN_VIEW_COURSE_avail_textbox.value = str(row[13])
        else:
          MAN_VIEW_COURSE_avail_textbox.value = ''
        if row[14] != None:
          MAN_VIEW_COURSE_wait_textbox.value = str(row[14])
        else:
          MAN_VIEW_COURSE_wait_textbox.value = ''
        if row[15] != None:
          MAN_VIEW_COURSE_desc_textbox.value = row[15]
        else:
          MAN_VIEW_COURSE_desc_textbox.value = ''
        if row[16] != None:
          MAN_VIEW_COURSE_enrollment_fee_textbox.value = str(row[16])
        else:
          MAN_VIEW_COURSE_enrollment_fee_textbox.value = ''
        if row[17] != None:
          MAN_VIEW_COURSE_startdate_datepicker.value = row[17].date()
        else:
          MAN_VIEW_COURSE_startdate_datepicker.value = ''
        if row[18] != None:
          MAN_VIEW_COURSE_enddate_datepicker.value = row[18].date()
        else:
          MAN_VIEW_COURSE_enddate_datepicker.value = ''
        MAN_VIEW_COURSE_status_dropdown.value = row[19]
  MAN_VIEW_COURSE_retrieve_button.on_click(on_MAN_VIEW_COURSE_retrieve_button_clicked)

  def on_MAN_VIEW_COURSE_clear_button_clicked(b):
    MAN_VIEW_COURSE_output.clear_output()
    MAN_VIEW_COURSE_id_textbox.value = ""
    MAN_VIEW_COURSE_INS_id_textbox.value = ""
    MAN_VIEW_COURSE_code_textbox.value = ""
    MAN_VIEW_COURSE_name_textbox.value = ""
    MAN_VIEW_COURSE_section_textbox.value = ""
    MAN_VIEW_COURSE_day_textbox.value = ""
    MAN_VIEW_COURSE_starttime_textbox.value = ""
    MAN_VIEW_COURSE_endtime_textbox.value = ""
    MAN_VIEW_COURSE_room_textbox.value = ""
    MAN_VIEW_COURSE_instructor_textbox.value = ""
    MAN_VIEW_COURSE_quota_textbox.value = ""
    MAN_VIEW_COURSE_enrol_textbox.value = ""
    MAN_VIEW_COURSE_avail_textbox.value = ""
    MAN_VIEW_COURSE_wait_textbox.value = ""
    MAN_VIEW_COURSE_desc_textbox.value = ""
    MAN_VIEW_COURSE_enrollment_fee_textbox.value = ""
    MAN_VIEW_COURSE_startdate_datepicker.value = ""
    MAN_VIEW_COURSE_enddate_datepicker.value = ""
    MAN_VIEW_COURSE_status_dropdown.value = ""
  MAN_VIEW_COURSE_clear_button.on_click(on_MAN_VIEW_COURSE_clear_button_clicked)

  courseview1HBox = widgets.HBox([MAN_VIEW_COURSE_dropdown, MAN_VIEW_COURSE_retrieve_button])
  courseview1aHBox = widgets.HBox([spacer])
  courseview2HBox = widgets.HBox([MAN_COURSE_id_label, MAN_VIEW_COURSE_id_textbox])
  courseview3HBox = widgets.HBox([MAN_COURSE_INS_id_label, MAN_VIEW_COURSE_INS_id_textbox, MAN_COURSE_code_label, MAN_VIEW_COURSE_code_textbox])
  courseview4HBox = widgets.HBox([MAN_COURSE_name_label, MAN_VIEW_COURSE_name_textbox, MAN_COURSE_credit_label, MAN_VIEW_COURSE_credit_textbox])
  courseview5HBox = widgets.HBox([MAN_COURSE_section_label, MAN_VIEW_COURSE_section_textbox, MAN_COURSE_day_label, MAN_VIEW_COURSE_day_textbox])
  courseview6HBox = widgets.HBox([MAN_COURSE_starttime_label, MAN_VIEW_COURSE_starttime_textbox, MAN_COURSE_endtime_label, MAN_VIEW_COURSE_endtime_textbox])
  courseview7HBox = widgets.HBox([MAN_COURSE_startdate_label, MAN_VIEW_COURSE_startdate_datepicker, MAN_COURSE_enddate_label, MAN_VIEW_COURSE_enddate_datepicker])
  courseview8HBox = widgets.HBox([MAN_COURSE_room_label, MAN_VIEW_COURSE_room_textbox, MAN_COURSE_instructor_label, MAN_VIEW_COURSE_instructor_textbox])
  courseview9HBox = widgets.HBox([MAN_COURSE_quota_label, MAN_VIEW_COURSE_quota_textbox, MAN_COURSE_enrol_label, MAN_VIEW_COURSE_enrol_textbox])
  courseview10HBox = widgets.HBox([MAN_COURSE_avail_label, MAN_VIEW_COURSE_avail_textbox, MAN_COURSE_wait_label, MAN_VIEW_COURSE_wait_textbox])
  courseview11HBox = widgets.HBox([MAN_COURSE_desc_label, MAN_VIEW_COURSE_desc_textbox, MAN_COURSE_enrollment_fee_label, MAN_VIEW_COURSE_enrollment_fee_textbox])
  courseview12HBox = widgets.HBox([MAN_COURSE_status_label, MAN_VIEW_COURSE_status_dropdown])
  courseview13HBox = widgets.HBox([MAN_VIEW_COURSE_output])

  display(widgets.VBox([courseview1HBox,
              courseview1aHBox,
              courseview2HBox,
              courseview3HBox,
              courseview4HBox,
              courseview5HBox,
              courseview6HBox,
              courseview7HBox,
              courseview8HBox,
              courseview9HBox,
              courseview10HBox,
              courseview11HBox,
              courseview12HBox,
              courseview13HBox]))
##################### View Course - End ####################

###########################################################
##################### Update/Delete Course (Ming)####################
###########################################################
with tb_m_course.output_to('Update/Delete Course'):
  MAN_MAIN_COURSE_output = widgets.Output()
  MAN_MAIN_COURSE_id_textbox = Text(disabled=True)
  MAN_MAIN_COURSE_INS_id_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_code_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_name_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_credit_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_section_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_day_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_starttime_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_endtime_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_room_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_instructor_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_quota_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_enrol_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_avail_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_wait_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_desc_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_enrollment_fee_textbox = Text(disabled=False)
  MAN_MAIN_COURSE_startdate_datepicker = DatePicker(disabled=False)
  MAN_MAIN_COURSE_enddate_datepicker = DatePicker(disabled=False)
  MAN_MAIN_COURSE_status_dropdown = Dropdown(disabled=False, options=['open', 'closed', 'cancelled'])

  #create buttons
  MAN_MAIN_COURSE_retrieve_button = widgets.Button(description='Retrieve')
  MAN_MAIN_COURSE_clear_button = widgets.Button(description='Clear')
  MAN_MAIN_COURSE_update_button = widgets.Button(description='Update')
  MAN_MAIN_COURSE_delete_button = widgets.Button(description='Delete')
  
  def on_MAN_MAIN_COURSE_retrieve_button_clicked(b):
    MAN_MAIN_COURSE_output.clear_output()
    if len(MAN_COURSE_id_list)==0:
      with MAN_MAIN_COURSE_output:
        display(widgets.Label('No course record found in Database!'))
    else:
      index_no = MAN_COURSE_list.index(MAN_MAIN_COURSE_dropdown.value)
      c.execute("""SELECT *
            FROM COURSE
            WHERE course_id = '""" + str(MAN_COURSE_id_list[index_no]) + "'")
      for row in c:
        MAN_MAIN_COURSE_id_textbox.value = str(row[0])
        MAN_MAIN_COURSE_INS_id_textbox.value = str(row[1])    # retrieve data from database and assign to the textbox
        MAN_MAIN_COURSE_code_textbox.value = row[2]
        MAN_MAIN_COURSE_name_textbox.value = row[3]
        MAN_MAIN_COURSE_credit_textbox.value = str(row[4])
        if row[5] != None:
          MAN_MAIN_COURSE_section_textbox.value = row[5]
        else:
          MAN_MAIN_COURSE_section_textbox.value = ''
        if row[6] != None:
          MAN_MAIN_COURSE_day_textbox.value = row[6]
        else:
          MAN_MAIN_COURSE_day_textbox.value = ''
        if row[7] != None:
          MAN_MAIN_COURSE_starttime_textbox.value = row[7]
        else:
          MAN_MAIN_COURSE_starttime_textbox.value = ''
        if row[8] != None:
          MAN_MAIN_COURSE_endtime_textbox.value = row[8]
        else:
          MAN_MAIN_COURSE_endtime_textbox.value = ''
        if row[9] != None:
          MAN_MAIN_COURSE_room_textbox.value = row[9]
        else:
          MAN_MAIN_COURSE_room_textbox.value = ''
        if row[10] != None:
          MAN_MAIN_COURSE_instructor_textbox.value = row[10]
        else:
          MAN_MAIN_COURSE_instructor_textbox.value = ''
        if row[11] != None:
          MAN_MAIN_COURSE_quota_textbox.value = str(row[11])
        else:
          MAN_MAIN_COURSE_quota_textbox.value = ''
        if row[12] != None:
          MAN_MAIN_COURSE_enrol_textbox.value = str(row[12])
        else:
          MAN_MAIN_COURSE_enrol_textbox.value = ''
        if row[13] != None:
          MAN_MAIN_COURSE_avail_textbox.value = str(row[13])
        else:
          MAN_MAIN_COURSE_avail_textbox.value = ''
        if row[14] != None:
          MAN_MAIN_COURSE_wait_textbox.value = str(row[14])
        else:
          MAN_MAIN_COURSE_wait_textbox.value = ''
        if row[15] != None:
          MAN_MAIN_COURSE_desc_textbox.value = row[15]
        else:
          MAN_MAIN_COURSE_desc_textbox.value = ''
        if row[16] != None:
          MAN_MAIN_COURSE_enrollment_fee_textbox.value = str(row[16])
        else:
          MAN_MAIN_COURSE_enrollment_fee_textbox.value = ''
        if row[17] != None:
          MAN_MAIN_COURSE_startdate_datepicker.value = row[17].date()
        else:
          MAN_MAIN_COURSE_startdate_datepicker.value = ''
        if row[18] != None:
          MAN_MAIN_COURSE_enddate_datepicker.value = row[18].date()
        else:
          MAN_MAIN_COURSE_enddate_datepicker.value = ''
        MAN_MAIN_COURSE_status_dropdown.value = row[19]
  MAN_MAIN_COURSE_retrieve_button.on_click(on_MAN_MAIN_COURSE_retrieve_button_clicked)

  def on_MAN_MAIN_COURSE_clear_button_clicked(b):
    MAN_MAIN_COURSE_output.clear_output()
    MAN_MAIN_COURSE_id_textbox.value = ""
    MAN_MAIN_COURSE_INS_id_textbox.value = ""
    MAN_MAIN_COURSE_code_textbox.value = ""
    MAN_MAIN_COURSE_name_textbox.value = ""
    MAN_MAIN_COURSE_credit_textbox.value = ""
    MAN_MAIN_COURSE_section_textbox.value = ""
    MAN_MAIN_COURSE_day_textbox.value = ""
    MAN_MAIN_COURSE_starttime_textbox.value = ""
    MAN_MAIN_COURSE_endtime_textbox.value = ""
    MAN_MAIN_COURSE_room_textbox.value = ""
    MAN_MAIN_COURSE_instructor_textbox.value = ""
    MAN_MAIN_COURSE_quota_textbox.value = ""
    MAN_MAIN_COURSE_enrol_textbox.value = ""
    MAN_MAIN_COURSE_avail_textbox.value = ""
    MAN_MAIN_COURSE_wait_textbox.value = ""
    MAN_MAIN_COURSE_desc_textbox.value = ""
    MAN_MAIN_COURSE_enrollment_fee_textbox.value = ""
  MAN_MAIN_COURSE_clear_button.on_click(on_MAN_MAIN_COURSE_clear_button_clicked)

  def MAN_MAIN_COURSE_check():
    pattern = r'^1[0-2]|0?[1-9]:[0-5][0-9][AP]M$'
    if MAN_MAIN_COURSE_id_textbox.value == "":
      print('Please select and retrieve a Course before Updating!')
    elif MAN_MAIN_COURSE_INS_id_textbox.value == "" or MAN_MAIN_COURSE_code_textbox.value == "" or MAN_MAIN_COURSE_name_textbox.value == "" or MAN_MAIN_COURSE_credit_textbox.value == "":
      print('Please input fields with "*" indicated!')
    elif int(MAN_MAIN_COURSE_credit_textbox.value) < 0 or int(MAN_MAIN_COURSE_credit_textbox.value) > 99:
      return('Credit must be >=0 and <=99 !')
    elif (MAN_MAIN_COURSE_starttime_textbox.value !="" and not(re.match(pattern, MAN_MAIN_COURSE_starttime_textbox.value))) or (MAN_MAIN_COURSE_endtime_textbox.value !="" and not(re.match(pattern, MAN_MAIN_COURSE_endtime_textbox.value))) :
      return('Start Time and End Time must be in ##:##_M format! e.g. 01:30PM, 12:05AM')
    elif (MAN_MAIN_COURSE_quota_textbox.value!='' or MAN_MAIN_COURSE_enrol_textbox.value!='' or MAN_MAIN_COURSE_avail_textbox.value!='' or MAN_MAIN_COURSE_wait_textbox.value!='') and (int(MAN_MAIN_COURSE_avail_textbox.value) != int(MAN_MAIN_COURSE_quota_textbox.value) - int(MAN_MAIN_COURSE_enrol_textbox.value)):
      return('Sum of the Available seats and Enrolled seats must not be larger than the Course Quota!')
    else:
      return(True)

  def on_MAN_MAIN_COURSE_update_button_clicked(b):
    MAN_MAIN_COURSE_output.clear_output()
    course_check = MAN_MAIN_COURSE_check()
    if course_check != True:
      with MAN_MAIN_COURSE_output:
        display(widgets.Label(course_check))
    else:
      c.execute(
      """UPDATE COURSE
        SET ins_id = {},
        course_code = '{}',
        course_name = '{}',
        course_credit = {},
        course_section = '{}',
        course_day = '{}',
        course_starttime = '{}',
        course_endtime = '{}',
        course_room = '{}',
        course_instructor = '{}',
        course_quota = {},
        course_enrol = {},
        course_avail = {},
        course_wait = {},
        course_desc = '{}',
        course_enrollment_fee = {},
        course_startdate = TO_DATE('{}', 'YYYY-MM-DD'),
        course_enddate = TO_DATE('{}', 'YYYY-MM-DD'),
        course_status = '{}'
        WHERE course_id = '{}'
      """.format(MAN_MAIN_COURSE_INS_id_textbox.value,
            MAN_MAIN_COURSE_code_textbox.value,
            MAN_MAIN_COURSE_name_textbox.value,
            MAN_MAIN_COURSE_credit_textbox.value,
            MAN_MAIN_COURSE_section_textbox.value,
            MAN_MAIN_COURSE_day_textbox.value,
            MAN_MAIN_COURSE_starttime_textbox.value,
            MAN_MAIN_COURSE_endtime_textbox.value,
            MAN_MAIN_COURSE_room_textbox.value,
            MAN_MAIN_COURSE_instructor_textbox.value,
            MAN_MAIN_COURSE_quota_textbox.value,
            MAN_MAIN_COURSE_enrol_textbox.value,
            MAN_MAIN_COURSE_avail_textbox.value,
            MAN_MAIN_COURSE_wait_textbox.value,
            MAN_MAIN_COURSE_desc_textbox.value,
            MAN_MAIN_COURSE_enrollment_fee_textbox.value,
            MAN_MAIN_COURSE_startdate_datepicker.value,
            MAN_MAIN_COURSE_enddate_datepicker.value,
            MAN_MAIN_COURSE_status_dropdown.value,
            MAN_MAIN_COURSE_id_textbox.value)
      )
      c.execute("commit")
      MAN_COURSE_LIST_GET()
      with MAN_MAIN_COURSE_output:
        display(widgets.Label('Update Success!'))
  MAN_MAIN_COURSE_update_button.on_click(on_MAN_MAIN_COURSE_update_button_clicked)

  def on_MAN_MAIN_COURSE_delete_button_clicked(b):
    MAN_MAIN_COURSE_output.clear_output()
    if MAN_MAIN_COURSE_id_textbox.value == "":
      print('Please select and retrieve a Course before Deleteing!')
    else:
      c.execute(
      """
            DELETE FROM COURSE 
            WHERE course_id = '{}'
      """.format(MAN_MAIN_COURSE_id_textbox.value)
      )
      c.execute("commit")
      MAN_MAIN_COURSE_clear_button.click()
      MAN_COURSE_LIST_GET()
      with MAN_MAIN_COURSE_output:
        display(widgets.Label('Delete Success!'))
  MAN_MAIN_COURSE_delete_button.on_click(on_MAN_MAIN_COURSE_delete_button_clicked)

  coursemain1HBox = widgets.HBox([MAN_MAIN_COURSE_dropdown, MAN_MAIN_COURSE_retrieve_button])
  coursemain1aHBox = widgets.HBox([spacer])
  coursemain2HBox = widgets.HBox([MAN_COURSE_id_label, MAN_MAIN_COURSE_id_textbox])
  coursemain3HBox = widgets.HBox([MAN_COURSE_INS_id_label, MAN_MAIN_COURSE_INS_id_textbox, MAN_COURSE_code_label, MAN_MAIN_COURSE_code_textbox])
  coursemain4HBox = widgets.HBox([MAN_COURSE_name_label, MAN_MAIN_COURSE_name_textbox, MAN_COURSE_credit_label, MAN_MAIN_COURSE_credit_textbox])
  coursemain5HBox = widgets.HBox([MAN_COURSE_section_label, MAN_MAIN_COURSE_section_textbox, MAN_COURSE_day_label, MAN_MAIN_COURSE_day_textbox])
  coursemain6HBox = widgets.HBox([MAN_COURSE_starttime_label, MAN_MAIN_COURSE_starttime_textbox, MAN_COURSE_endtime_label, MAN_MAIN_COURSE_endtime_textbox])
  coursemain7HBox = widgets.HBox([MAN_COURSE_startdate_label, MAN_MAIN_COURSE_startdate_datepicker, MAN_COURSE_enddate_label, MAN_MAIN_COURSE_enddate_datepicker])
  coursemain8HBox = widgets.HBox([MAN_COURSE_room_label, MAN_MAIN_COURSE_room_textbox, MAN_COURSE_instructor_label, MAN_MAIN_COURSE_instructor_textbox])
  coursemain9HBox = widgets.HBox([MAN_COURSE_quota_label, MAN_MAIN_COURSE_quota_textbox, MAN_COURSE_enrol_label, MAN_MAIN_COURSE_enrol_textbox])
  coursemain10HBox = widgets.HBox([MAN_COURSE_avail_label, MAN_MAIN_COURSE_avail_textbox, MAN_COURSE_wait_label, MAN_MAIN_COURSE_wait_textbox])
  coursemain11HBox = widgets.HBox([MAN_COURSE_desc_label, MAN_MAIN_COURSE_desc_textbox, MAN_COURSE_enrollment_fee_label, MAN_MAIN_COURSE_enrollment_fee_textbox])
  coursemain11aHBox = widgets.HBox([MAN_COURSE_status_label, MAN_MAIN_COURSE_status_dropdown])
  coursemain12HBox = widgets.HBox([MAN_MAIN_COURSE_update_button, MAN_MAIN_COURSE_delete_button])
  coursemain13HBox = widgets.HBox([MAN_MAIN_COURSE_clear_button])
  coursemain14HBox = widgets.HBox([MAN_MAIN_COURSE_output])

  display(widgets.VBox([coursemain1HBox,
            coursemain1aHBox,
            coursemain2HBox,
            coursemain3HBox,
            coursemain4HBox,
            coursemain5HBox,
            coursemain6HBox,
            coursemain7HBox,
            coursemain8HBox,
            coursemain9HBox,
            coursemain10HBox,
            coursemain11HBox,
            coursemain11aHBox,
            coursemain12HBox,
            coursemain13HBox,
            coursemain14HBox]))  
##################### Update/Delete Course - End ####################

###########################################################
##################### Insert Course (Ming)####################
###########################################################
with tb_m_course.output_to('Insert Course'):
  #create textbox
  MAN_INS_COURSE_output = widgets.Output()

  MAN_INS_COURSE_INS_id_textbox = Text(disabled=False)
  MAN_INS_COURSE_code_textbox = Text(disabled=False)
  MAN_INS_COURSE_name_textbox = Text(disabled=False)
  MAN_INS_COURSE_credit_textbox = Text(disabled=False)
  MAN_INS_COURSE_section_textbox = Text(disabled=False)
  MAN_INS_COURSE_day_textbox = Text(disabled=False)
  MAN_INS_COURSE_starttime_textbox = Text(disabled=False)
  MAN_INS_COURSE_endtime_textbox = Text(disabled=False)
  MAN_INS_COURSE_room_textbox = Text(disabled=False)
  MAN_INS_COURSE_instructor_textbox = Text(disabled=False)
  MAN_INS_COURSE_quota_textbox = Text(disabled=False)
  MAN_INS_COURSE_enrol_textbox = Text(disabled=False)
  MAN_INS_COURSE_avail_textbox = Text(disabled=False)
  MAN_INS_COURSE_wait_textbox = Text(disabled=False)
  MAN_INS_COURSE_desc_textbox = Text(disabled=False)
  MAN_INS_COURSE_enrollment_fee_textbox = Text(disabled=False)
  MAN_INS_COURSE_startdate_datepicker = DatePicker(disabled=False)
  MAN_INS_COURSE_enddate_datepicker = DatePicker(disabled=False)
  MAN_INS_COURSE_status_dropdown = Dropdown(disabled=False, options=['open', 'closed', 'cancelled'])

  #create buttons 
  MAN_INS_COURSE_insert_button = widgets.Button(description='Insert')
  MAN_INS_COURSE_clear_button = widgets.Button(description='Clear')

  def MAN_INS_COURSE_check():
    pattern = r'^1[0-2]|0?[1-9]:[0-5][0-9][AP]M$'
    if MAN_INS_COURSE_INS_id_textbox.value == "" or MAN_INS_COURSE_code_textbox.value == "" or MAN_INS_COURSE_name_textbox.value == "" or MAN_INS_COURSE_credit_textbox.value == "":
      return('Please input fields with "*" indicated!')
    elif int(MAN_INS_COURSE_credit_textbox.value) < 0 or int(MAN_INS_COURSE_credit_textbox.value) > 99:
      return('Credit must be >=0 and <=99 !')
    elif (MAN_INS_COURSE_starttime_textbox!="" and not(re.match(pattern, MAN_INS_COURSE_starttime_textbox.value))) or (MAN_INS_COURSE_endtime_textbox!="" and not(re.match(pattern, MAN_INS_COURSE_endtime_textbox.value))):
      return('Start Time and End Time must be in ##:##_M format! e.g. 01:30PM, 12:05AM')
    elif (MAN_INS_COURSE_quota_textbox.value!="" or MAN_INS_COURSE_enrol_textbox.value!="" or MAN_INS_COURSE_avail_textbox.value!="" or MAN_INS_COURSE_wait_textbox.value!="") and (int(MAN_INS_COURSE_avail_textbox.value) != int(MAN_INS_COURSE_quota_textbox.value) - int(MAN_INS_COURSE_enrol_textbox.value)):
      return('Sum of the Available seats and Enrolled seats must not be larger than the Course Quota!')
    else:
      return(True)

  def on_MAN_INS_COURSE_insert_button_clicked(b):
    MAN_INS_COURSE_output.clear_output()
    course_check = MAN_INS_COURSE_check()
    if course_check != True:
      with MAN_INS_COURSE_output:
        display(widgets.Label(course_check))
    else:
      #get max id
      c.execute("""SELECT max(COURSE_ID) FROM COURSE""")
      for row in c:
        if row[0] != None:
          course_id_max = row[0]
        else:
          course_id_max = 0
      #insert
      c.execute("""INSERT INTO COURSE VALUES ({},
                          '{}',
                          '{}',
                          {},
                          '{}',
                          '{}',
                          '{}',
                          '{}',
                          '{}',
                          '{}',
                          '{}',
                          {},
                          {},
                          {},
                          {},
                          '{}',
                          {},
                          TO_DATE('{}', 'YYYY-MM-DD'),
                          TO_DATE('{}', 'YYYY-MM-DD'),
                          '{}'
      """.format(course_id_max+1,
            MAN_INS_COURSE_INS_id_textbox.value,
            MAN_INS_COURSE_code_textbox.value,
            MAN_INS_COURSE_name_textbox.value,
            MAN_INS_COURSE_credit_textbox.value,
            MAN_INS_COURSE_section_textbox.value,
            MAN_INS_COURSE_day_textbox.value,
            MAN_INS_COURSE_starttime_textbox.value,
            MAN_INS_COURSE_endtime_textbox.value,
            MAN_INS_COURSE_room_textbox.value,
            MAN_INS_COURSE_instructor_textbox.value,
            MAN_INS_COURSE_quota_textbox.value,
            MAN_INS_COURSE_enrol_textbox.value,
            MAN_INS_COURSE_avail_textbox.value,
            MAN_INS_COURSE_wait_textbox.value,
            MAN_INS_COURSE_desc_textbox.value,
            MAN_INS_COURSE_enrollment_fee_textbox.value,
            MAN_INS_COURSE_startdate_datepicker.value,
            MAN_INS_COURSE_enddate_datepicker.value,
            MAN_INS_COURSE_status_dropdown.value)
      )
      c.execute("commit")
      MAN_INS_COURSE_clear_button.click()
      MAN_COURSE_LIST_GET()
      with MAN_INS_COURSE_output:
        display(widgets.label('Insert Success!'))

  MAN_INS_COURSE_insert_button.on_click(on_MAN_INS_COURSE_insert_button_clicked)

  def on_MAN_INS_COURSE_clear_button_clicked(b):
    MAN_INS_COURSE_output.clear_output()
    MAN_INS_COURSE_INS_id_textbox.value = ""
    MAN_INS_COURSE_code_textbox.value = ""
    MAN_INS_COURSE_name_textbox.value = ""
    MAN_INS_COURSE_credit_textbox.value = ""
    MAN_INS_COURSE_section_textbox.value = ""
    MAN_INS_COURSE_day_textbox.value = ""
    MAN_INS_COURSE_starttime_textbox.value = ""
    MAN_INS_COURSE_endtime_textbox.value = ""
    MAN_INS_COURSE_room_textbox.value = ""
    MAN_INS_COURSE_instructor_textbox.value = ""
    MAN_INS_COURSE_quota_textbox.value = ""
    MAN_INS_COURSE_enrol_textbox.value = ""
    MAN_INS_COURSE_avail_textbox.value = ""
    MAN_INS_COURSE_wait_textbox.value = ""
    MAN_INS_COURSE_desc_textbox.value = ""
    MAN_INS_COURSE_enrollment_fee_textbox.value = ""
  MAN_INS_COURSE_clear_button.on_click(on_MAN_INS_COURSE_clear_button_clicked)

  courseins1HBox = widgets.HBox([MAN_COURSE_INS_id_label, MAN_INS_COURSE_INS_id_textbox, MAN_COURSE_code_label, MAN_INS_COURSE_code_textbox])
  courseins2HBox = widgets.HBox([MAN_COURSE_name_label, MAN_INS_COURSE_name_textbox, MAN_COURSE_credit_label, MAN_INS_COURSE_credit_textbox])
  courseins3HBox = widgets.HBox([MAN_COURSE_section_label, MAN_INS_COURSE_section_textbox, MAN_COURSE_day_label, MAN_INS_COURSE_day_textbox])
  courseins4HBox = widgets.HBox([MAN_COURSE_starttime_label, MAN_INS_COURSE_starttime_textbox, MAN_COURSE_endtime_label, MAN_INS_COURSE_endtime_textbox])
  courseins4aHBox = widgets.HBox([MAN_COURSE_startdate_label, MAN_INS_COURSE_startdate_datepicker, MAN_COURSE_enddate_label, MAN_INS_COURSE_enddate_datepicker])
  courseins5HBox = widgets.HBox([MAN_COURSE_room_label, MAN_INS_COURSE_room_textbox, MAN_COURSE_instructor_label, MAN_INS_COURSE_instructor_textbox])
  courseins6HBox = widgets.HBox([MAN_COURSE_quota_label, MAN_INS_COURSE_quota_textbox, MAN_COURSE_enrol_label, MAN_INS_COURSE_enrol_textbox])
  courseins7HBox = widgets.HBox([MAN_COURSE_avail_label, MAN_INS_COURSE_avail_textbox, MAN_COURSE_wait_label, MAN_INS_COURSE_wait_textbox])
  courseins8HBox = widgets.HBox([MAN_COURSE_desc_label, MAN_INS_COURSE_desc_textbox, MAN_COURSE_enrollment_fee_label, MAN_INS_COURSE_enrollment_fee_textbox])
  courseins9HBox = widgets.HBox([MAN_COURSE_status_label, MAN_INS_COURSE_status_dropdown])
  courseins10HBox = widgets.HBox([MAN_INS_COURSE_insert_button, MAN_INS_COURSE_clear_button])
  courseins11HBox = widgets.HBox([MAN_INS_COURSE_output])

  display(widgets.VBox([courseins1HBox,
            courseins2HBox,
            courseins3HBox,
            courseins4HBox,
            courseins4aHBox,
            courseins5HBox,
            courseins6HBox,
            courseins7HBox,
            courseins8HBox,
            courseins9HBox,
            courseins10HBox,
            courseins11HBox]))  

##################### Insert Course - End ####################


###########################################################
##################### Roster (Ming)####################
##########################################################
with tb_m_course.output_to('Generate Roster'):
  #Create output area
  MAN_COURSE_roster_output = widgets.Output()

  #create lists to store each column of data
  MAN_COURSE_roster_stu_id_all = []
  MAN_COURSE_roster_stu_name_all = []
  MAN_COURSE_roster_stu_email_all = []
  MAN_COURSE_roster_stu_job_all = []
  MAN_COURSE_roster_stu_qualification_all = []
  MAN_studentinfo_HBox_all = []

  #Create a list store all lists
  MAN_COURSE_roster_stu_all = [MAN_COURSE_roster_stu_id_all, MAN_COURSE_roster_stu_name_all, MAN_COURSE_roster_stu_email_all, MAN_COURSE_roster_stu_job_all, MAN_COURSE_roster_stu_qualification_all, MAN_studentinfo_HBox_all]
  
  def MAN_COURSE_roster_generate(b):
    #Clear output area
    MAN_COURSE_roster_output.clear_output()
    #Clear all lists
    for ls in MAN_COURSE_roster_stu_all:
      ls.clear()
    
    #Get selected course id
    try:
      index_no = MAN_COURSE_list.index(MAN_ROSTER_COURSE_dropdown.value)
    except:
      with MAN_COURSE_roster_output:
        display(widgets.Label('There are no course and roster cannot be generated!'))
    else:
      MAN_COURSE_roster_course_id = str(MAN_COURSE_id_list[index_no])

      c.execute("""SELECT s.STU_ID, s.STU_NAME, s.STU_EMAIL, s.STU_JOB, s.STU_QUALIFICATION
            FROM LA205.STUDENT s
            JOIN LA205.ENROLLMENT e ON s.STU_ID = e.STU_ID
            WHERE e.COURSE_ID = '""" + MAN_COURSE_roster_course_id + "'")
      
      for row in c:
        MAN_COURSE_roster_stu_id_all.append(Text(str(row[0]), disabled = True))
        MAN_COURSE_roster_stu_name_all.append(Text(row[1], disabled = True))
        MAN_COURSE_roster_stu_email_all.append(Text(row[2], disabled = True))
        MAN_COURSE_roster_stu_job_all.append(Text(row[3], disabled = True))
        MAN_COURSE_roster_stu_qualification_all.append(Text(row[4], disabled = True))

      header = widgets.HBox([Text('Student ID', disabled=True), Text('Name', disabled=True), Text('Email', disabled=True), Text('Job', disabled=True), Text('Qualification', disabled=True)])
      
      for i in range(len(MAN_COURSE_roster_stu_id_all)):
        row_studentinfo_HBox = widgets.HBox([MAN_COURSE_roster_stu_id_all[i], MAN_COURSE_roster_stu_name_all[i], MAN_COURSE_roster_stu_email_all[i], MAN_COURSE_roster_stu_job_all[i], MAN_COURSE_roster_stu_qualification_all[i]])
        MAN_studentinfo_HBox_all.append(row_studentinfo_HBox)
      with MAN_COURSE_roster_output:
        if len(MAN_COURSE_roster_stu_id_all) ==0:
          display(widgets.Label('No student has enrolled in the selected course!'))
        else:
          display(header, widgets.VBox([row for row in MAN_studentinfo_HBox_all]))
  
  #create button
  MAN_COURSE_ROSTER_generate_button = widgets.Button(description='Generate')
  MAN_COURSE_ROSTER_generate_button.on_click(MAN_COURSE_roster_generate)

  mancourseroster1HBox = widgets.HBox([MAN_ROSTER_COURSE_dropdown, MAN_COURSE_ROSTER_generate_button])
  display(mancourseroster1HBox, MAN_COURSE_roster_output)

##################### Roster - End ####################
##################### Course - End ####################
##################### Manager Home - End ####################

###########################################################
##################### Instructor Home (Ming)####################
##########################################################

###########################################################
##################### Account Info (Ming)####################
##########################################################
with tb_ins.output_to('Account Information'):
  #Create Textbox
  INST_INFO_id_textbox = Text(disabled=True)
  INST_INFO_name_textbox = Text(disabled=True)
  INST_INFO_username_textbox = Text(disabled=True)
  INST_INFO_email_textbox = Text(disabled=True)
  INST_INFO_edu_bg_textbox = Text(disabled=True)
  INST_INFO_bio_textbox = Text(disabled=True)

  #create labels
  INST_INFO_layout = Layout(width='160px')

  INST_INFO_id_label = Label('Instructor ID:', layout=INST_INFO_layout)
  INST_INFO_name_label = Label('Name:', layout=INST_INFO_layout)
  INST_INFO_username_label = Label('Username:', layout=INST_INFO_layout)
  INST_INFO_email_label = Label('Email:', layout=INST_INFO_layout)
  INST_INFO_edu_bg_label = Label('Educational Background:', layout=INST_INFO_layout)
  INST_INFO_bio_label = Label('Biography:', layout=INST_INFO_layout)

  def INST_INFO_GET():
    c.execute("""SELECT INS_ID,
              INS_NAME,
              INS_USERNAME,
              INS_EMAIL,
              INS_EDUCATION_BG,
              INS_BIOGRAPHY
          FROM INSTRUCTOR
          WHERE INS_ID = """ + str(id))
    for row in c:
      INST_INFO_id_textbox.value = str(row[0])
      INST_INFO_name_textbox.value = row[1]
      INST_INFO_username_textbox.value = row[2]
      INST_INFO_email_textbox.value = row[3]
      try:
        INST_INFO_edu_bg_textbox.value = row[4]
      except:
        INST_INFO_edu_bg_textbox.value = ''
      try:
        INST_INFO_bio_textbox.value = row[5]
      except:
        INST_INFO_bio_textbox.value = ''

  instinfo1HBox = widgets.HBox([INST_INFO_id_label, INST_INFO_id_textbox])
  instinfo2HBox = widgets.HBox([INST_INFO_name_label, INST_INFO_name_textbox])
  instinfo3HBox = widgets.HBox([INST_INFO_username_label, INST_INFO_username_textbox])
  instinfo4HBox = widgets.HBox([INST_INFO_email_label, INST_INFO_email_textbox])
  instinfo5HBox = widgets.HBox([INST_INFO_edu_bg_label, INST_INFO_edu_bg_textbox])
  instinfo6HBox = widgets.HBox([INST_INFO_bio_label, INST_INFO_bio_textbox])

  display(instinfo1HBox, instinfo2HBox, instinfo3HBox, instinfo4HBox, instinfo5HBox, instinfo6HBox)

##################### Account Info - End ####################

###########################################################
##################### Course (Ming)####################
##########################################################

###########################################################
##################### View Course (Ming)####################
##########################################################
with tb_ins_course.output_to("Course"):
  INST_COURSE_output = widgets.Output()
  #create textbox
  INST_COURSE_id_textbox = Text(disabled=True)
  INST_COURSE_INS_id_textbox = Text(disabled=True)
  INST_COURSE_code_textbox = Text(disabled=True)
  INST_COURSE_name_textbox = Text(disabled=True)
  INST_COURSE_credit_textbox = Text(disabled=True)
  INST_COURSE_section_textbox = Text(disabled=True)
  INST_COURSE_day_textbox = Text(disabled=True)
  INST_COURSE_starttime_textbox = Text(disabled=True)
  INST_COURSE_endtime_textbox = Text(disabled=True)
  INST_COURSE_room_textbox = Text(disabled=True)
  INST_COURSE_instructor_textbox = Text(disabled=True)
  INST_COURSE_quota_textbox = Text(disabled=True)
  INST_COURSE_enrol_textbox = Text(disabled=True)
  INST_COURSE_avail_textbox = Text(disabled=True)
  INST_COURSE_wait_textbox = Text(disabled=True)
  INST_COURSE_desc_textbox = Text(disabled=True)
  INST_COURSE_enrollment_fee_textbox = Text(disabled=True)
  INST_COURSE_startdate_datepicker = DatePicker(disabled=True)
  INST_COURSE_enddate_datepicker = DatePicker(disabled=True)
  INST_COURSE_status_dropdown = Dropdown(options=['open', 'closed', 'cancelled'], disabled=True)

  #create dropdown
  INST_COURSE_dropdown = widgets.Dropdown()
  INST_COURSE_roster_dropdown = widgets.Dropdown()
  INST_COURSE_review_dropdown = widgets.Dropdown()

  #create button
  INST_COURSE_retrieve_button = widgets.Button(description='Retrieve', disabled=False)

  # INST_id = 12345678 #To be removed

  INST_id_textbox = Text(disabled=True)

  #create list
  INST_COURSE_list = [] 
  INST_COURSE_id_list = []

  def INST_COURSE_GET():
    INST_id_textbox.value = str(id)
    global INST_COURSE_list, INST_COURSE_id_list
    INST_COURSE_list = [] 
    INST_COURSE_id_list = []
    #get course list
    c.execute("""SELECT course_id,
              course_code,
              course_name,
              course_section
          FROM COURSE
          WHERE INS_ID = '""" + INST_id_textbox.value + "'" +
        """ORDER BY course_code, course_section""")
    for row in c:
      INST_COURSE_list.append(str(row[1]) + '('+ row[3] + ') - ' + row[2] + ' (' + str(row[0]) +')')
      INST_COURSE_id_list.append(row[0])
    #Update dropdown menu
    INST_COURSE_dropdown.options = INST_COURSE_list
    INST_COURSE_roster_dropdown.options = INST_COURSE_list
    INST_COURSE_review_dropdown.options = INST_COURSE_list


  def on_INST_COURSE_retrieve_button_clicked(b):
    INST_COURSE_output.clear_output()
    if len(INST_COURSE_id_list)==0:
      with INST_COURSE_output:
        display(widgets.Label('No course record found in Database!'))
    else:
      index_no = INST_COURSE_list.index(INST_COURSE_dropdown.value)
      c.execute("""SELECT *
            FROM COURSE
            WHERE course_id = '""" + str(INST_COURSE_id_list[index_no]) + "'")
      # retrieve data from database and assign to the textbox
      for row in c:
        INST_COURSE_id_textbox.value = str(row[0])
        INST_COURSE_code_textbox.value = row[2]
        INST_COURSE_name_textbox.value = row[3]
        INST_COURSE_credit_textbox.value = str(row[4])
        if row[5] != None:
          INST_COURSE_section_textbox.value = row[5]
        else:
          INST_COURSE_section_textbox.value = ''
        if row[6] != None:
          INST_COURSE_day_textbox.value = row[6]
        else:
          INST_COURSE_day_textbox.value = ''
        if row[7] != None:
          INST_COURSE_starttime_textbox.value = row[7]
        else:
          INST_COURSE_starttime_textbox.value = ''
        if row[8] != None:
          INST_COURSE_endtime_textbox.value = row[8]
        else:
          INST_COURSE_endtime_textbox.value = ''
        if row[9] != None:
          INST_COURSE_room_textbox.value = row[9]
        else:
          INST_COURSE_room_textbox.value = ''
        if row[10] != None:
          INST_COURSE_instructor_textbox.value = row[10]
        else:
          INST_COURSE_instructor_textbox.value = ''
        if row[11] != None:
          INST_COURSE_quota_textbox.value = str(row[11])
        else:
          INST_COURSE_quota_textbox.value = ''
        if row[12] != None:
          INST_COURSE_enrol_textbox.value = str(row[12])
        else:
          INST_COURSE_enrol_textbox.value = ''
        if row[13] != None:
          INST_COURSE_avail_textbox.value = str(row[13])
        else:
          INST_COURSE_avail_textbox.value = ''
        if row[14] != None:
          INST_COURSE_wait_textbox.value = str(row[14])
        else:
          INST_COURSE_wait_textbox.value = ''
        if row[15] != None:
          INST_COURSE_desc_textbox.value = row[15]
        else:
          INST_COURSE_desc_textbox.value = ''
        if row[16] != None:
          INST_COURSE_enrollment_fee_textbox.value = str(row[16])
        else:
          INST_COURSE_enrollment_fee_textbox.value = ''
        if row[17] != None:
          INST_COURSE_startdate_datepicker.value = row[17].date()
        else:
          INST_COURSE_startdate_datepicker.value = ''
        if row[18] != None:
          INST_COURSE_enddate_datepicker.value = row[18].date()
        else:
          INST_COURSE_enddate_datepicker.value = ''
        INST_COURSE_status_dropdown.value = row[19]
  INST_COURSE_retrieve_button.on_click(on_INST_COURSE_retrieve_button_clicked)

  instcourse1HBox = widgets.HBox([INST_COURSE_dropdown, INST_COURSE_retrieve_button])
  instcourse2HBox = widgets.HBox([spacer])
  instcourse3HBox = widgets.HBox([MAN_COURSE_INS_id_label, INST_id_textbox])
  instcourse4HBox = widgets.HBox([MAN_COURSE_id_label, INST_COURSE_id_textbox])
  instcourse5HBox = widgets.HBox([MAN_COURSE_code_label, INST_COURSE_code_textbox, MAN_COURSE_name_label, INST_COURSE_name_textbox])
  instcourse6HBox = widgets.HBox([MAN_COURSE_credit_label, INST_COURSE_credit_textbox])
  instcourse7HBox = widgets.HBox([MAN_COURSE_section_label, INST_COURSE_section_textbox, MAN_COURSE_day_label, INST_COURSE_day_textbox])
  instcourse8HBox = widgets.HBox([MAN_COURSE_starttime_label, INST_COURSE_starttime_textbox, MAN_COURSE_endtime_label, INST_COURSE_endtime_textbox])
  instcourse9HBox = widgets.HBox([MAN_COURSE_startdate_label, INST_COURSE_startdate_datepicker, MAN_COURSE_enddate_label, INST_COURSE_enddate_datepicker])
  instcourse10HBox = widgets.HBox([MAN_COURSE_room_label, INST_COURSE_room_textbox, MAN_COURSE_instructor_label, INST_COURSE_instructor_textbox])
  instcourse11HBox = widgets.HBox([MAN_COURSE_quota_label, INST_COURSE_quota_textbox, MAN_COURSE_enrol_label, INST_COURSE_enrol_textbox])
  instcourse12HBox = widgets.HBox([MAN_COURSE_avail_label, INST_COURSE_avail_textbox, MAN_COURSE_wait_label, INST_COURSE_wait_textbox])
  instcourse13HBox = widgets.HBox([MAN_COURSE_desc_label, INST_COURSE_desc_textbox, MAN_COURSE_enrollment_fee_label, INST_COURSE_enrollment_fee_textbox])
  instcourse14HBox = widgets.HBox([INST_COURSE_output])

  display(widgets.VBox([instcourse1HBox,
            instcourse2HBox,
            instcourse3HBox,
            instcourse4HBox,
            instcourse5HBox,
            instcourse6HBox,
            instcourse7HBox,
            instcourse8HBox,
            instcourse9HBox,
            instcourse10HBox,
            instcourse11HBox,
            instcourse12HBox,
            instcourse13HBox,
            instcourse14HBox]))  
  
###########################################################
######################## Review (Ming)#######################
##########################################################
with tb_ins_course.output_to('Review'):
  INST_COURSE_review_output = widgets.Output()
  #create button
  INST_COURSE_review_retrieve_button = widgets.Button(description='Retrieve')
  INST_COURSE_review_firstpage_button = widgets.Button(description='First Page')
  INST_COURSE_review_nextpage_button = widgets.Button(description='Next Page')
  INST_COURSE_review_prevpage_button = widgets.Button(description='Previous Page')
  INST_COURSE_review_lastpage_button = widgets.Button(description='Last Page')

  #create label
  INST_COURSE_label_layout = Layout(width='100px')
  INST_COURSE_review_avg_rating_label = Label("Average Rating:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_feedback_labelA = Label("Feedback 1:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_feedback_labelB = Label("Feedback 2:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_feedback_labelC = Label("Feedback 3:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_feedback_labelD = Label("Feedback 4:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_feedback_labelE = Label("Feedback 5:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_rating_labelA = Label("Rating 1:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_rating_labelB = Label("Rating 2:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_rating_labelC = Label("Rating 3:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_rating_labelD = Label("Rating 4:", layout = INST_COURSE_label_layout)
  INST_COURSE_review_rating_labelE = Label("Rating 5:", layout = INST_COURSE_label_layout)

  #Create page number
  INST_COURSE_review_page = 1
  INST_COURSE_review_max_page = 1

  #Create list
  INST_COURSE_review_review_list = []
  INST_COURSE_review_rating_list = []

  #create textbox
  INST_COURSE_review_avg_rating_textbox = Text(disabled=True)
  INST_COURSE_review_feedback_textbox1 = Text(disabled=True)
  INST_COURSE_review_rating_textbox1 = Text(disabled=True)
  INST_COURSE_review_feedback_textbox2 = Text(disabled=True)
  INST_COURSE_review_rating_textbox2 = Text(disabled=True)
  INST_COURSE_review_feedback_textbox3 = Text(disabled=True)
  INST_COURSE_review_rating_textbox3 = Text(disabled=True)
  INST_COURSE_review_feedback_textbox4 = Text(disabled=True)
  INST_COURSE_review_rating_textbox4 = Text(disabled=True)
  INST_COURSE_review_feedback_textbox5 = Text(disabled=True)
  INST_COURSE_review_rating_textbox5 = Text(disabled=True)

  #Function
  def update_INST_COURSE_review_feedback_labelA(change):
    INST_COURSE_review_feedback_labelA.value = str(change.new)
  def update_INST_COURSE_review_feedback_labelB(change):
    INST_COURSE_review_feedback_labelB.value = str(change.new)
  def update_INST_COURSE_review_feedback_labelC(change):
    INST_COURSE_review_feedback_labelC.value = str(change.new)
  def update_INST_COURSE_review_feedback_labelD(change):
    INST_COURSE_review_feedback_labelD.value = str(change.new)
  def update_INST_COURSE_review_feedback_labelE(change):
    INST_COURSE_review_feedback_labelE.value = str(change.new)

  def update_INST_COURSE_review_rating_labelA(change):
    INST_COURSE_review_rating_labelA.value = str(change.new)
  def update_INST_COURSE_review_rating_labelB(change):
    INST_COURSE_review_rating_labelB.value = str(change.new)
  def update_INST_COURSE_review_rating_labelC(change):
    INST_COURSE_review_rating_labelC.value = str(change.new)
  def update_INST_COURSE_review_rating_labelD(change):
    INST_COURSE_review_rating_labelD.value = str(change.new)
  def update_INST_COURSE_review_rating_labelE(change):
    INST_COURSE_review_rating_labelE.value = str(change.new)
  
  INST_COURSE_review_feedback_labelA.observe(update_INST_COURSE_review_feedback_labelA, names='value')
  INST_COURSE_review_feedback_labelB.observe(update_INST_COURSE_review_feedback_labelB, names='value')
  INST_COURSE_review_feedback_labelC.observe(update_INST_COURSE_review_feedback_labelC, names='value')
  INST_COURSE_review_feedback_labelD.observe(update_INST_COURSE_review_feedback_labelD, names='value')
  INST_COURSE_review_feedback_labelE.observe(update_INST_COURSE_review_feedback_labelE, names='value')

  INST_COURSE_review_rating_labelA.observe(update_INST_COURSE_review_rating_labelA, names='value')
  INST_COURSE_review_rating_labelB.observe(update_INST_COURSE_review_rating_labelB, names='value')
  INST_COURSE_review_rating_labelC.observe(update_INST_COURSE_review_rating_labelC, names='value')
  INST_COURSE_review_rating_labelD.observe(update_INST_COURSE_review_rating_labelD, names='value')
  INST_COURSE_review_rating_labelE.observe(update_INST_COURSE_review_rating_labelE, names='value')
  
  def on_INST_COURSE_review_retrieve_button_clicked(b):
    INST_COURSE_review_output.clear_output()
    global INST_COURSE_review_review_list, INST_COURSE_review_rating_list
    global INST_COURSE_review_max_page
    #Put FB_Comment and FB_Rating as lists separately
    try:
      index_no = INST_COURSE_list.index(INST_COURSE_review_dropdown.value)
    except:
      with INST_COURSE_review_output:
        display(widgets.Label('There are no course and no feedback and rating can be retrieved!'))
    else:
      c.execute("""SELECT FB_COMMENT,
                FB_RATING
          FROM FEEDBACK
          WHERE course_id = '""" + str(INST_COURSE_id_list[index_no]) + "'")

      INST_COURSE_review_review_list = []
      INST_COURSE_review_rating_list = []

      for row in c:
        INST_COURSE_review_review_list.append(row[0])
        INST_COURSE_review_rating_list.append(row[1])

      #get max page
      INST_COURSE_review_max_page = math.ceil(len(INST_COURSE_review_review_list)/5)

      #Get avg rating
      c.execute("SELECT AVG(FB_RATING) FROM FEEDBACK WHERE course_id = '" + str(INST_COURSE_id_list[index_no]) + "'")
      for row in c:
        if row[0] != None:
          INST_COURSE_review_avg_rating_textbox.value = str(row[0])
        else:
          INST_COURSE_review_avg_rating_textbox.value = 'N/A'
      INST_COURSE_review_display()
  INST_COURSE_review_retrieve_button.on_click(on_INST_COURSE_review_retrieve_button_clicked)

  def INST_COURSE_review_display():
    #display result with check None or blank
    pageA = 1+5*(INST_COURSE_review_page-1)
  #Set A
    try:
      INST_COURSE_review_feedback_textbox1.value = str(INST_COURSE_review_review_list[pageA-1])
    except:
      INST_COURSE_review_feedback_textbox1.value = 'N/A'
    finally:
      INST_COURSE_review_feedback_labelA.value = f'Feedback {pageA}:'

    try:
      INST_COURSE_review_rating_textbox1.value = str(INST_COURSE_review_rating_list[pageA-1])
    except:
      INST_COURSE_review_rating_textbox1.value = 'N/A'
    finally:
      INST_COURSE_review_rating_labelA.value = f'Rating {pageA}:'
  #Set B
    try:
      INST_COURSE_review_feedback_textbox2.value = str(INST_COURSE_review_review_list[pageA])
    except:
      INST_COURSE_review_feedback_textbox2.value = 'N/A'
    finally:
      INST_COURSE_review_feedback_labelB.value = f'Feedback {pageA+1}:'

    try:
      INST_COURSE_review_rating_textbox2.value = str(INST_COURSE_review_rating_list[pageA])
    except:
      INST_COURSE_review_rating_textbox2.value = 'N/A'
    finally:
      INST_COURSE_review_rating_labelB.value = f'Rating {pageA+1}:'
  #Set C
    try:
      INST_COURSE_review_feedback_textbox3.value = str(INST_COURSE_review_review_list[pageA+1])
    except:
      INST_COURSE_review_feedback_textbox3.value = 'N/A'
    finally:
      INST_COURSE_review_feedback_labelC.value = f'Feedback {pageA+2}:'

    try:
      INST_COURSE_review_rating_textbox3.value = str(INST_COURSE_review_rating_list[pageA+1])
    except:
      INST_COURSE_review_rating_textbox3.value = 'N/A'
    finally:
      INST_COURSE_review_rating_labelC.value = f'Rating {pageA+2}:'
  #Set D
    try:
      INST_COURSE_review_feedback_textbox4.value = str(INST_COURSE_review_review_list[pageA+2])
    except:
      INST_COURSE_review_feedback_textbox4.value = 'N/A'
    finally:
      INST_COURSE_review_feedback_labelD.value = f'Feedback {pageA+3}:'

    try:
      INST_COURSE_review_rating_textbox4.value = str(INST_COURSE_review_rating_list[pageA+2])
    except:
      INST_COURSE_review_rating_textbox4.value = 'N/A'
    finally:
      INST_COURSE_review_rating_labelD.value = f'Rating {pageA+3}:'
  #Set E
    try:
      INST_COURSE_review_feedback_textbox5.value = str(INST_COURSE_review_review_list[pageA+3])
    except:
      INST_COURSE_review_feedback_textbox5.value = 'N/A'
    finally:
      INST_COURSE_review_feedback_labelE.value = f'Feedback {pageA+4}:'

    try:
      INST_COURSE_review_rating_textbox5.value = str(INST_COURSE_review_rating_list[pageA+3])
    except:
      INST_COURSE_review_rating_textbox5.value = 'N/A'
    finally:
      INST_COURSE_review_rating_labelE.value = f'Rating {pageA+4}:'

  #functions for page buttons
  def on_INST_COURSE_review_prevpage_button_clicked(b):
    INST_COURSE_review_output.clear_output()
    global INST_COURSE_review_page
    if INST_COURSE_review_page != 1:
      INST_COURSE_review_page = INST_COURSE_review_page - 1
      INST_COURSE_review_display()
    else:
      with INST_COURSE_review_output:
        display(widgets.Label('You are already on the First Page!'))
  INST_COURSE_review_prevpage_button.on_click(on_INST_COURSE_review_prevpage_button_clicked)

  def on_INST_COURSE_review_nextpage_button_clicked(b):
    INST_COURSE_review_output.clear_output()
    global INST_COURSE_review_page
    if INST_COURSE_review_page < INST_COURSE_review_max_page:
      INST_COURSE_review_page = INST_COURSE_review_page + 1
      INST_COURSE_review_display()
    else:
      with INST_COURSE_review_output:
        display(widgets.Label('You are already on the Last Page!'))
  INST_COURSE_review_nextpage_button.on_click(on_INST_COURSE_review_nextpage_button_clicked)

  def on_INST_COURSE_review_firstpage_button_clicked(b):
    INST_COURSE_review_output.clear_output()
    global INST_COURSE_review_page
    if INST_COURSE_review_page != 1:
      INST_COURSE_review_page = 1
    else:
      with INST_COURSE_review_output:
        display(widgets.Label('You are already on the First Page!'))
    INST_COURSE_review_display()
  INST_COURSE_review_firstpage_button.on_click(on_INST_COURSE_review_firstpage_button_clicked)

  def on_INST_COURSE_review_lastpage_button_clicked(b):
    INST_COURSE_review_output.clear_output()
    global INST_COURSE_review_page
    if INST_COURSE_review_max_page == 0 or INST_COURSE_review_page == INST_COURSE_review_max_page:
      with INST_COURSE_review_output:
        display(widgets.Label('You are already on the Last Page!'))
    else:
      INST_COURSE_review_page = INST_COURSE_review_max_page
    INST_COURSE_review_display()
  
  INST_COURSE_review_lastpage_button.on_click(on_INST_COURSE_review_lastpage_button_clicked)
  instcourserewview1HBox = widgets.HBox([INST_COURSE_review_dropdown, INST_COURSE_review_retrieve_button])
  instcourserewview2HBox = widgets.HBox([INST_COURSE_review_avg_rating_label, INST_COURSE_review_avg_rating_textbox])
  instcourserewview3HBox = widgets.HBox([INST_COURSE_review_feedback_labelA, INST_COURSE_review_feedback_textbox1, INST_COURSE_review_rating_labelA, INST_COURSE_review_feedback_textbox1])
  instcourserewview4HBox = widgets.HBox([INST_COURSE_review_feedback_labelB, INST_COURSE_review_feedback_textbox2, INST_COURSE_review_rating_labelB, INST_COURSE_review_feedback_textbox2])
  instcourserewview5HBox = widgets.HBox([INST_COURSE_review_feedback_labelC, INST_COURSE_review_feedback_textbox3, INST_COURSE_review_rating_labelC, INST_COURSE_review_feedback_textbox3])
  instcourserewview6HBox = widgets.HBox([INST_COURSE_review_feedback_labelD, INST_COURSE_review_feedback_textbox4, INST_COURSE_review_rating_labelD, INST_COURSE_review_feedback_textbox4])
  instcourserewview7HBox = widgets.HBox([INST_COURSE_review_feedback_labelE, INST_COURSE_review_feedback_textbox5, INST_COURSE_review_rating_labelE, INST_COURSE_review_feedback_textbox5])
  instcourserewview8HBox = widgets.HBox([INST_COURSE_review_firstpage_button, INST_COURSE_review_prevpage_button, INST_COURSE_review_nextpage_button, INST_COURSE_review_lastpage_button])
  instcoursereview9HBox = widgets.HBox([INST_COURSE_review_output])

  display(widgets.VBox([instcourserewview1HBox,
            instcourserewview2HBox,
            instcourserewview3HBox,
            instcourserewview4HBox,
            instcourserewview5HBox,
            instcourserewview6HBox,
            instcourserewview7HBox,
            instcourserewview8HBox,
            instcoursereview9HBox]))

##################### Review - End ####################

###########################################################
##################### Class Roster (Ming)####################
##########################################################
with tb_ins_course.output_to('Class Roster'):
  #Create output area
  INST_COURSE_roster_output = widgets.Output()

  #create lists to store each column of data
  INST_COURSE_roster_stu_id_all = []
  INST_COURSE_roster_stu_name_all = []
  INST_COURSE_roster_stu_email_all = []
  INST_COURSE_roster_stu_job_all = []
  INST_COURSE_roster_stu_qualification_all = []
  INST_COURSE_roster_studentinfo_HBox_all = []

  #Create a list store all lists
  INST_COURSE_roster_stu_all = [INST_COURSE_roster_stu_id_all, INST_COURSE_roster_stu_name_all, INST_COURSE_roster_stu_email_all, INST_COURSE_roster_stu_job_all, INST_COURSE_roster_stu_qualification_all, INST_COURSE_roster_studentinfo_HBox_all]

  def INST_COURSE_roster_generate(b):
    #clear output area
    INST_COURSE_roster_output.clear_output()
    #clear all lists
    for ls in INST_COURSE_roster_stu_all:
      ls.clear()
    #get selected course id
    try:
      index_no = INST_COURSE_list.index(INST_COURSE_roster_dropdown.value)
    except:
      with INST_COURSE_roster_output:
        display(widgets.Label('There are no course and roster cannot be generated!'))
    else:
      INST_COURSE_roster_course_id = INST_COURSE_id_list[index_no]
    #Get roster and put in a list
      c.execute("""SELECT s.STU_ID, s.STU_NAME, s.STU_EMAIL, s.STU_JOB, s.STU_QUALIFICATION
            FROM LA205.STUDENT s
            JOIN LA205.ENROLLMENT e ON s.STU_ID = e.STU_ID
            WHERE e.COURSE_ID = '""" + str(INST_COURSE_roster_course_id) + "'")
      for row in c:
        INST_COURSE_roster_stu_id_all.append(Text(str(row[0]), disabled = True))
        INST_COURSE_roster_stu_name_all.append(Text(row[1], disabled = True))
        INST_COURSE_roster_stu_email_all.append(Text(row[2], disabled = True))
        INST_COURSE_roster_stu_job_all.append(Text(row[3], disabled = True))
        INST_COURSE_roster_stu_qualification_all.append(Text(row[4], disabled = True))

      header = widgets.HBox([Text('Student ID', disabled=True), Text('Name', disabled=True), Text('Email', disabled=True), Text('Job', disabled=True), Text('Qualification', disabled=True)])
      for i in range(len(INST_COURSE_roster_stu_id_all)):
        row_studentinfo_HBox = widgets.HBox([INST_COURSE_roster_stu_id_all[i], INST_COURSE_roster_stu_name_all[i], INST_COURSE_roster_stu_email_all[i], INST_COURSE_roster_stu_job_all[i], INST_COURSE_roster_stu_qualification_all[i]])
        INST_COURSE_roster_studentinfo_HBox_all.append(row_studentinfo_HBox)
      with INST_COURSE_roster_output:
        if len(INST_COURSE_roster_stu_id_all) == 0:
          display(widgets.Label('No student has enrolled in the selected course!'))
        else:
          display(header, widgets.VBox([row for row in INST_COURSE_roster_studentinfo_HBox_all]))
    #create button
  INST_COURSE_roster_generate_button = widgets.Button(description='Generate')
  INST_COURSE_roster_generate_button.on_click(INST_COURSE_roster_generate)

  instcourseroster1HBox = widgets.HBox([MAN_COURSE_INS_id_label, INST_id_textbox])
  instcourseroster2HBox = widgets.HBox([INST_COURSE_roster_dropdown, INST_COURSE_roster_generate_button])
  display(instcourseroster1HBox, instcourseroster2HBox, INST_COURSE_roster_output)

##################### Roster - End ####################

##################### Course - End ####################

##################### Instructor - End ####################

###########################################################
##################### Refund Student list ####################
##########################################################
with tb_m.output_to('Refund List'):
  RET_id_textbox = Text(value='', placeholder='Enter a Student ID', description='Student ID:', disabled=False)
  RET_course_textbox = Text(value='', placeholder='Enter a Course ID', description='Course ID:', disabled=False)


# create 5 buttons and 1 label
  ret_all_button = widgets.Button(description="Retrieve All")
  ret_all_clear_button = widgets.Button(description="Clear All")
  ret_all_back_button = widgets.Button(description="Back")
  ret_all_refund_button = widgets.Button(description="Refund")
  ret_all_spacer = widgets.Label("--------------------------------------------------------------------------")
  ret_all_out = widgets.Output()


# create 5 lists for containing Textboxes and 1 list for containing HBoxes
  ret_all_id_textboxes  = []
  ret_all_cid_textboxes = []
  ret_all_name_textboxes = []
  ret_all_amount_textboxes = []
  ret_all_HBoxes = []
  ret_all_status_textboxes = []
  ret_all_enr_textboxes = []
  ret_all_stbox = []
  labels = widgets.Label(value='Enter the Student_ID and Course_ID to update record after refund:')


# define the event handlers for the buttons when they are being clicked
  def on_ret_all_refund_button_clicked(b):
    c.execute(
        """
          SELECT STU_ID, COURSE_ID
          FROM ENROLLMENT
          WHERE  ENR_STATUS = 'waiting for refund'
        """
    )
    for row in c:
      ret_all_id_textboxes.append(Text(value=str(row[0]), description='Student ID:', disabled=True)) 
      ret_all_cid_textboxes.append(Text(value=str(row[1]), description='Course ID:', disabled=True)) 


    x = False
    for i in range(len(ret_all_id_textboxes)):
      if str(RET_id_textbox.value) == ret_all_id_textboxes[i].value and str(RET_course_textbox.value) == ret_all_cid_textboxes[i].value :
        c.execute(
            """
              UPDATE ENROLLMENT 
              SET ENR_STATUS = 'refunded'
              WHERE STU_ID = '{}' AND COURSE_ID = '{}'
            """.format(RET_id_textbox.value, RET_course_textbox.value)
        )
        c.execute("commit")
        x = True
        break
      else:
        x = False

    if x:
      with ret_all_out:
        print("Update Success!")
        on_ret_all_button_clicked(b)

    else:
      with ret_all_out:
        print('Invalid Entry, Please re-enter the Student ID and Couse ID')

  ret_all_refund_button.on_click(on_ret_all_refund_button_clicked)



  def on_ret_all_button_clicked(b):
    on_ret_all_clear_button_clicked(b)
    line1HBox = widgets.HBox([RET_id_textbox ,RET_course_textbox ,ret_all_refund_button])
    c.execute(
        """
          SELECT STU_ID, COURSE_ID, ENR_STATUS
          FROM ENROLLMENT
          WHERE  ENR_STATUS = 'waiting for refund'
        """
    )
    for row in c:
      ret_all_id_textboxes.append(Text(value=str(row[0]), description='Student ID:', disabled=True)) 
      ret_all_cid_textboxes.append(Text(value=str(row[1]), description='Course ID:', disabled=True)) 
      ret_all_status_textboxes.append(Text(value=str(row[2]), description='Course Status:', disabled=True)) 
 
    for i in range(len(ret_all_id_textboxes)):
      ret_all_HBoxes.append( widgets.HBox(  [ret_all_id_textboxes[i], ret_all_cid_textboxes[i], ret_all_status_textboxes[i]] ) )

    ret_all_VBoxes = widgets.VBox([i for i in ret_all_HBoxes])

    with ret_all_out:
      display(ret_all_VBoxes)
  ret_all_button.on_click(on_ret_all_button_clicked)



  def on_ret_all_clear_button_clicked(b):
    ret_all_id_textboxes.clear()
    ret_all_id_textboxes.clear()
    ret_all_name_textboxes.clear()
    ret_all_amount_textboxes.clear()

    ret_all_HBoxes.clear()
    ret_all_out.clear_output()
  ret_all_clear_button.on_click(on_ret_all_clear_button_clicked)

# display the objects
  line1HBox = widgets.HBox([RET_id_textbox ,RET_course_textbox ,ret_all_refund_button])
  ret_all_control_HBox = widgets.HBox([ret_all_button, ret_all_clear_button])
  ret_all_VBox = widgets.VBox([ret_all_control_HBox, ret_all_spacer, ret_all_out, labels ,line1HBox])
  display(ret_all_VBox)


################ Refund List - End ################


###########################################################
##################### Manager Feedback ####################
##########################################################
with tb_m_feedback.output_to('View Feedback'):

# create 
  course_textbox = Text(value='', placeholder='Enter a Course ID', description='Course ID:', disabled=False)
  feedback_retrieve_all_button = widgets.Button(description="Retrieve All")
  feedback_clear_button = widgets.Button(description="Clear All")
  feedback_back_button = widgets.Button(description="Back")
  feedback_retrieve_button = widgets.Button(description="Retrieve")
  feedback_spacer = widgets.Label("--------------------------------------------------------------------------")
  feedback_out = widgets.Output()

# create 5 lists for containing Textboxes and 1 list for containing HBoxes
  feedback_course_id_textboxes = []
  feedback_id_textboxes = []
  feedback_rating_textboxes = []
  feedback_HBox = []
  feedback_textboxes = []
  feedback_ret_rate = []



# define the event handlers for the buttons when they are being clicked
  def feedback_retrieve_all_button_clicked(b):
    feedback_clear_button_clicked(b)

    c.execute(
        """
        SELECT * FROM FEEDBACK
        ORDER BY COURSE_ID
        """)
    for row in c:
      feedback_course_id_textboxes.append(Text(value=str(row[2]), description='COURSE ID:', disabled=True)) 
      feedback_id_textboxes.append(Text(value=str(row[0]), description='FB ID:', disabled=True)) 
      feedback_rating_textboxes.append(Text(value=str(row[4]), description='RATING:', disabled=True))

    for i in range(len(feedback_id_textboxes)):
      feedback_HBox.append( widgets.HBox(  [feedback_course_id_textboxes[i], feedback_id_textboxes[i], feedback_rating_textboxes[i]]  ) )

    with feedback_out:
      display(widgets.VBox([i for i in feedback_HBox]))
  feedback_retrieve_all_button.on_click(feedback_retrieve_all_button_clicked)

  def feedback_clear_button_clicked(b):
    feedback_course_id_textboxes.clear()
    feedback_id_textboxes.clear()
    feedback_rating_textboxes.clear()
    feedback_HBox.clear()
    feedback_textboxes.clear()

    feedback_HBox.clear()
    feedback_out.clear_output()
    feedback_clear_button.on_click(feedback_clear_button_clicked)

  def feedback_retrieve_button_clicked(b):
    feedback_clear_button_clicked(b)

    c.execute("SELECT * FROM FEEDBACK WHERE COURSE_ID = '" + course_textbox.value + "'")
    feedback_found = False

    for row in c:
      feedback_course_id_textboxes.append(Text(value=str(row[2]), description='COURSE ID:', disabled=True)) 
      feedback_id_textboxes.append(Text(value=str(row[0]), description='FB ID:', disabled=True)) 
      feedback_rating_textboxes.append(Text(value=str(row[4]), description='RATING:', disabled=True))
      feedback_found = True

    for i in range(len(feedback_id_textboxes)):
        feedback_HBox.append( widgets.HBox(  [feedback_course_id_textboxes[i], feedback_id_textboxes[i], feedback_rating_textboxes[i]]  ) )

    if feedback_found:
      with feedback_out:
        display(widgets.VBox([i for i in feedback_HBox]))
    else:
      with feedback_out:
        print('   Course Not Found, please re-enter the Course ID to search for Feedbacks')

  feedback_retrieve_button.on_click(feedback_retrieve_button_clicked)

  def feedback_back_button_clicked(b):
    showPage("Main Menu")
  feedback_back_button.on_click(feedback_back_button_clicked)




# display the objects
  line1HBox = widgets.HBox([course_textbox, feedback_retrieve_all_button])
  feedback_control_HBox = widgets.HBox([feedback_retrieve_all_button, feedback_clear_button, course_textbox, feedback_retrieve_button])
  feedback_VBox = widgets.VBox([feedback_control_HBox, feedback_spacer, feedback_out, feedback_back_button])
  display(feedback_VBox)


################ Manager Feedback - End ################

showPage('Login')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>